<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 2 - Phase 1 - eyamrog

The aim of this phase is to web scrape the archive of mainstream journals in the areas of knowledge targeted in this study.

## Required Python packages

- beautifulsoup4
- lxml
- pandas
- requests
- selenium
- tqdm

## Importing the required libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
import sys
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Defining input variables

In [2]:
input_directory = 'cl_st2_input_eyamrog'
output_directory = 'cl_st2_ph1_eyamrog'

## Creating output directory

In [3]:
# Check if the output directory already exists. If it does, do nothing. If it doesn't exist, create it.
if os.path.exists(output_directory):
    print('Output directory already exists.')
else:
    try:
        os.makedirs(output_directory)
        print('Output directory successfully created.')
    except OSError as e:
        print('Failed to create the directory:', e)
        sys.exit(1)

Output directory already exists.


## Inspecting the articles of the `chatgpt_erpp_pp` corpus per area of knowledge

### Importing the data into a DataFrame

In [4]:
df_scielo_preprint_preChatGPT_en = pd.read_json(f'{input_directory}/scielo_chatgpt_erpp_pp.jsonl', lines=True)

In [5]:
df_scielo_preprint_preChatGPT_en.dtypes

Title                    object
URL                      object
Authors                  object
Published                object
PDF Language             object
PDF URL                  object
Submitted                 int64
Posted                    int64
Text                     object
Text ID                  object
Area of Knowledge        object
Text Paragraphs          object
Text Paragraphs Count     int64
Text ChatGPT             object
dtype: object

In [6]:
df_scielo_preprint_preChatGPT_en['Submitted'] = pd.to_datetime(df_scielo_preprint_preChatGPT_en['Submitted'], unit='ms')
df_scielo_preprint_preChatGPT_en['Posted'] = pd.to_datetime(df_scielo_preprint_preChatGPT_en['Posted'], unit='ms')

In [7]:
df_scielo_preprint_preChatGPT_en

,Title,URL,Authors,Published,PDF Language,PDF URL,Submitted,Posted,Text,Text ID,Area of Knowledge,Text Paragraphs,Text Paragraphs Count,Text ChatGPT
0,"(Fern flora of Viçosa, Minas Gerais State, Bra...",https://preprints.scielo.org/index.php/scielo/...,"Nelson Túlio Lage Pena, Pedro Bond Schwartsburd",Submitted 11/22/2022 - Posted 11/23/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-22,2022-11-23,Publication status: Preprint has been publishe...,t000000,Biological Sciences,"(Fern flora of Viçosa, Minas Gerais State, Bra...",29,In this research article focusing on the fern ...
1,Assembling the perfect bacterial genome using ...,https://preprints.scielo.org/index.php/scielo/...,"Ryan R. Wick, Louise M. Judd, Kathryn E. Holt",Submitted 11/11/2022 - Posted 11/11/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-11,2022-11-11,Publication status: Preprint has been publishe...,t000001,Biological Sciences,A perfect bacterial genome assembly is one whe...,31,A flawless bacterial genome assembly is charac...
2,ON METHODOLOGY AND METHODS FOR ANALYSING CLASS...,https://preprints.scielo.org/index.php/scielo/...,Leonardo Goncalves Lago,Submitted 11/10/2022 - Posted 11/16/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-10,2022-11-16,Publication status: Preprint has been publishe...,t000002,Applied Social Sciences,This article presents a theoretical work whose...,65,This article presents a theoretical work with ...
3,"MOBILIZING LINGUISTIC AND SEMIOTIC RESOURCES, ...",https://preprints.scielo.org/index.php/scielo/...,"Estêvão Cabral, Marylin Martin-Jones",Submitted 11/07/2022 - Posted 11/07/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-07,2022-11-07,Publication status: Preprint has been submitte...,t000003,"Linguistic, literature and arts",This paper is based on research of a socioling...,58,This paper presents a sociolinguistic and ethn...
4,PORTUGUESE AO PÉ DO BERIMBAU: ON CAPOEIRA AS A...,https://preprints.scielo.org/index.php/scielo/...,"Mike Baynham, Jolana Hanusova",Submitted 11/04/2022 - Posted 11/04/2022,PDF,https://preprints.scielo.org/index.php/scielo/...,2022-11-04,2022-11-04,Publication status: Preprint has been submitte...,t000004,"Linguistic, literature and arts",From its historical origins as a resistant and...,123,Originating as a defiant and violently suppres...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Challenges in the fight against the COVID-19 p...,https://preprints.scielo.org/index.php/scielo/...,Eduardo Alexandrino Servolo Medeiros,Submitted 04/15/2020 - Posted 04/15/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-15,2020-04-15,*Corresponding author. E-mail: edubalaccih@gma...,t000311,Health Sciences,We are living the most important pandemic in r...,9,"The ongoing global pandemic, triggered by the ..."
312,Information about the new coronavirus disease ...,https://preprints.scielo.org/index.php/scielo/...,Claudio Márcio Amaral de Oliveira Lima,Submitted 04/13/2020 - Posted 04/13/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-13,2020-04-13,V\nRadiol Bras. 2020 Mar/Abr;53(2):V–VI\n0100-...,t000312,Health Sciences,"Coronavirus is a zoonotic virus, an RNA virus ...",9,The coronavirus is a zoonotic RNA virus belong...
313,ACE2 diversity in placental mammals reveals th...,https://preprints.scielo.org/index.php/scielo/...,"Bibiana Sampaio de Oliveira Fam, Pedro Vargas-...",Submitted 04/11/2020 - Posted 04/28/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-11,2020-04-28,Status: Preprint has been published in a journ...,t000313,Biological Sciences,The recent emergence of SARS-CoV-2 is responsi...,35,The recent emergence of SARS-CoV-2 has led to ...
314,Coronavirus 2: Analysis of Regularity of Compl...,https://preprints.scielo.org/index.php/scielo/...,Yuri Morales-López,Submitted 04/10/2020 - Posted 06/04/2020,PDF,https://preprints.scielo.org/index.php/scielo/...,2020-04-10,20

### Agricultural Sciences

In [8]:
df_agricultural_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Agricultural Sciences']
df_agricultural_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_agricultural_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Applied Social Sciences

In [9]:
df_applied_social_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Applied Social Sciences']
df_applied_social_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_applied_social_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Biological Sciences

In [10]:
df_biological_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Biological Sciences']
df_biological_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_biological_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Engineering

In [11]:
df_engineering = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Engineering']
df_engineering[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_engineering.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Exact and Earth Sciences

In [12]:
df_exact_earth_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Exact and Earth Sciences']
df_exact_earth_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/exact_earth_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Health Sciences

In [13]:
df_health_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Health Sciences']
df_health_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_health_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Human Sciences

In [14]:
df_human_sciences = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Human Sciences']
df_human_sciences[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_human_sciences.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

### Linguistics, literature and arts

In [15]:
df_linguistics_literature_arts = df_scielo_preprint_preChatGPT_en[df_scielo_preprint_preChatGPT_en['Area of Knowledge'] == 'Linguistic, literature and arts']
df_linguistics_literature_arts[['Text ID', 'Text Paragraphs Count', 'Title']].to_csv(f'{output_directory}/scielo_linguistics_literature_arts.tsv', sep='\t', index=False, encoding='utf-8', lineterminator='\n')

## Agricultural Sciences

This discipline has been excluded from the study.

### Scraping [Annual Review of Plant Biology](https://www.annualreviews.org/content/journals/arplant)

#### Capturing a few document pages for inspection

In [3]:
url_sample_1 = 'https://www.annualreviews.org/content/journals/arplant/71/1'
url_sample_11 = 'https://www.annualreviews.org/content/journals/10.1146/annurev-arplant-081519-035846'

In [4]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'google_esf')))
document_page_sample_1 = driver.page_source

with open(f'{output_directory}/ar_plant_biology_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_element_located((By.ID, 'google_esf')))
document_page_sample_11 = driver.page_source

with open(f'{output_directory}/ar_plant_biology_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [5]:
def scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url}{i}"
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.ID, 'google_esf')))
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')

        articles = soup.find_all('div', class_='articleMetadata js-articleMetadata')

        for article in articles:
            title_tag = article.find('span', class_='articleTitle js-articleTitle title').find('a')
            title = title_tag.get_text(strip=True) if title_tag else ''
            article_url = root_url + title_tag['href'] if title_tag else ''

            authors_tags = article.find_all('span', class_='author-list__item')
            authors = ', '.join(tag.get_text(strip=True) for tag in authors_tags)

            vol_year_range = article.find('div', class_='volYearPageRange')
            vol_year = vol_year_range.get_text(strip=True) if vol_year_range else ''

            doi_tag = article.find('div', class_='articleSourceTag').find('a')
            doi = doi_tag['href'] if doi_tag else ''

            data.append({
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Year/Page Range': vol_year,
                'DOI': doi,
                'Area of Knowledge': area_of_knowledge
            })

    driver.quit()

    return pd.DataFrame(data)

##### Volume 73, 2022

In [6]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/arplant/73/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Agricultural Sciences'

In [7]:
df_2022 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


##### Volume 72, 2021

In [8]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/arplant/72/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Agricultural Sciences'

In [9]:
df_2021 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:08<00:00,  4.35s/it]


##### Volume 71, 2020

In [10]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/arplant/71/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Agricultural Sciences'

In [11]:
df_2020 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:08<00:00,  4.12s/it]


##### Concatenating the DataFrames

In [12]:
df_ar_plant_biology = pd.concat([df_2022, df_2021, df_2020], ignore_index=True)
df_ar_plant_biology

,Title,URL,Authors,Vol/Year/Page Range,DOI,Area of Knowledge
0,"Adventures in Life and Science, from Light to ...",https://www.annualreviews.org/content/journals...,Elaine Tobin,"Vol. 73 \n(2022),pp.1–16",https://doi.org/10.1146/annurev-arplant-090921...,Agricultural Sciences
1,Phosphorus Acquisition and Utilization in Plants,https://www.annualreviews.org/content/journals...,Hans Lambers,"Vol. 73 \n(2022),pp.17–42",https://doi.org/10.1146/annurev-arplant-102720...,Agricultural Sciences
2,Meeting in the Middle: Lessons and Opportuniti...,https://www.annualreviews.org/content/journals...,"Mae Antonette Mercado, , andAnthony J. Studer","Vol. 73 \n(2022),pp.43–65",https://doi.org/10.1146/annurev-arplant-102720...,Agricultural Sciences
3,Plant Proteome Dynamics,https://www.annualreviews.org/content/journals...,"Julia Mergner, , andBernhard Kuster","Vol. 73 \n(2022),pp.67–92",https://doi.org/10.1146/annurev-arplant-102620...,Agricultural Sciences
4,Evolution and Functions of Plant U-Box Protein...,https://www.annualreviews.org/content/journals...,"Jana Trenner, ,Jacqueline Monaghan, ,Bushra Sa...","Vol. 73 \n(2022),pp.93–121",https://doi.org/10.1146/annurev-arplant-102720...,Agricultural Sciences
...,...,...,...,...,...,...
83,Phenotyping: New Windows into the Plant for Br...,https://www.annualreviews.org/content/journals...,"Michelle Watt, ,Fabio Fiorani, ,Björn Usadel, ...","Vol. 71 \n(2020),pp.689–712",https://doi.org/10.1146/annurev-arplant-042916...,Agricultural Sciences
84,The Genomics ofCannabisand Its Close Relatives,https://www.annualreviews.org/content/journals...,"I. Kovalchuk, ,M. Pellino, ,P. Rigault, ,R. va...","Vol. 71 \n(2020),pp.713–739",https://doi.org/10.1146/annurev-arplant-081519...,Agricultural Sciences
85,Sequencing and Analyzing the Transcriptomes of...,https://www.annualreviews.org/content/journals...,"Gane Ka-Shu Wong, ,Douglas E. Soltis, ,Jim Lee...","Vol. 71 \n(2020),pp.741–765",https://doi.org/10.1146/annurev-arplant-042916...,Agricultural Sciences
86,Engineering Synthetic Signaling in Plants,https://www.annualreviews.org/content/journals...,"Alexander R. Leydon, ,Hardik P. Gala, ,Sarah G...","Vol. 71 \n(2020),pp.767–788",https://doi.org/10.1146/annurev-arplant-081519...,Agricultural Sciences


#### Inspecting a few articles

In [13]:
df_ar_plant_biology.at[0, 'URL']

'https://www.annualreviews.org/content/journals/10.1146/annurev-arplant-090921-091346'

#### Exporting to a file

In [14]:
df_ar_plant_biology.to_json(f"{output_directory}/ar_plant_biology.jsonl", orient='records', lines=True)

### Scraping [Nature Food](https://www.nature.com/natfood/)

#### Capturing a few document pages for inspection

In [16]:
url_sample_1 = 'https://www.nature.com/natfood/research-articles?searchType=journalSearch&sort=PubDate&type=article&page=1'
url_sample_11 = 'https://www.nature.com/articles/s43016-024-01039-1'

In [17]:
document_page_sample_1 = requests.get(url_sample_1)
document_page_sample_11 = requests.get(url_sample_11)

In [18]:
with open(f'{output_directory}/natfood_sample1.html', 'wb') as file:
    file.write(document_page_sample_1.content)

with open(f'{output_directory}/natfood_sample11.html', 'wb') as file:
    file.write(document_page_sample_11.content)

#### Scraping the articles

In [19]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/natfood/research-articles?searchType=journalSearch&sort=PubDate&type=article&page='
start_page = 1
end_page = 15
area_of_knowledge = 'Agricultural Sciences'

In [20]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_food = pd.DataFrame(data)

100%|██████████| 15/15 [00:18<00:00,  1.22s/it]


In [21]:
df_nature_food['Published'] = pd.to_datetime(df_nature_food['Published'])
df_nature_food

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Potassium limits productivity in intensive cer...,https://www.nature.com/articles/s43016-024-010...,"Gonzalo Rizzo, Fahmuddin Agus, Patricio Grassini",2024-10-22,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
1,Proteomic signatures of healthy dietary patter...,https://www.nature.com/articles/s43016-024-010...,"Kai Zhu, Rui Li, Gang Liu",2024-09-27,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
2,Localized nitrogen management strategies can h...,https://www.nature.com/articles/s43016-024-010...,"Yize Liu, Minghao Zhuang, Yan Hao",2024-09-27,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
3,Human activities shape important geographic di...,https://www.nature.com/articles/s43016-024-010...,"Yuping Xiang, Guangliang Liu, Guibin Jiang",2024-09-26,https://www.nature.com/articles/s43016-024-010...,,Agricultural Sciences
4,Enhanced agricultural carbon sinks provide ben...,https://www.nature.com/articles/s43016-024-010...,"Stefan Frank, Andrey Lessa Derci Augustynczik,...",2024-09-23,https://www.nature.com/articles/s43016-024-010...,Open Access,Agricultural Sciences
...,...,...,...,...,...,...,...
277,Improved performance ofEimeria-infected chicke...,https://www.nature.com/articles/s43016-020-0029-6,"Philip A. Lessard, Matthew Parker, R. Michael ...",2020-02-10,https://www.nature.com/articles/s43016-020-002...,,Agricultural Sciences
278,Multidimensional characterization of global fo...,https://www.nature.com/articles/s43016-019-0012-2,"James Bentham, Gitanjali M Singh, Majid Ezzati",2020-01-13,https://www.nature.com/articles/s43016-019-001...,,Agricultural Sciences
279,Identification and stacking of crucial traits ...,https://www.nature.com/articles/s43016-019-0007-z,"Ratan Chopra, Evan B. Johnson, M. David Marks",2020-01-13,https://www.nature.com/articles/s43016-019-000...,,Agricultural Sciences
280,Shifts in national land use and food productio...,https://www.nature.com/articles/s43016-019-0011-3,"Paul D. L. Ritchie, Greg S. Smith, Ian J. Bateman",2020-01-13,https://www.nature.com/articles/s43016-019-001...,,Agricultural Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [22]:
df_nature_food_open_access = df_nature_food[df_nature_food['Open Access'] == 'Open Access']
df_nature_food_open_access = df_nature_food_open_access[df_nature_food_open_access['Published'] <= '2022-11-30']
df_nature_food_open_access = df_nature_food_open_access.reset_index(drop=True)
df_nature_food_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Transfer of cannabinoids into the milk of dair...,https://www.nature.com/articles/s43016-022-006...,"Bettina Wagner, Pietro Gerletti, Robert Pieper",2022-11-14,https://www.nature.com/articles/s43016-022-006...,Open Access,Agricultural Sciences
1,A systematic scoping review of the sustainabil...,https://www.nature.com/articles/s43016-022-006...,"A. Charlotte Bunge, Amanda Wood, Line J. Gordon",2022-11-03,https://www.nature.com/articles/s43016-022-006...,Open Access,Agricultural Sciences
2,Rights and representation support justice acro...,https://www.nature.com/articles/s43016-022-006...,"Christina C. Hicks, Jessica A. Gephart, Rosamo...",2022-10-18,https://www.nature.com/articles/s43016-022-006...,Open Access,Agricultural Sciences
3,Global dietary quality in 185 countries from 1...,https://www.nature.com/articles/s43016-022-005...,"Victoria Miller, Patrick Webb, Rubina Hakeem",2022-09-19,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
4,Food system by-products upcycled in livestock ...,https://www.nature.com/articles/s43016-022-005...,"Vilma Sandström, Anna Chrysafi, Matti Kummu",2022-09-19,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
5,Global food insecurity and famine from reduced...,https://www.nature.com/articles/s43016-022-005...,"Lili Xia, Alan Robock, Ryan Heneghan",2022-08-15,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
6,A shift from cattle to camel and goat farming ...,https://www.nature.com/articles/s43016-022-005...,"Jaber Rahimi, Erwann Fillol, Klaus Butterbach-...",2022-07-21,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
7,Time management governs climate resilience and...,https://www.nature.com/articles/s43016-022-005...,"Andrew J. McDonald, Balwinder-Singh, R. K. Malik",2022-07-21,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
8,A hybrid inorganic–biological artificial photo...,https://www.nature.com/articles/s43016-022-005...,"Elizabeth C. Hann, Sean Overa, Robert E. Jinke...",2022-06-23,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences
9,A systematic review of the definitions and int...,https://www.nature.com/articles/s43016-022-005...,"Kajsa Resare Sahlin, Joanna Trewern",2022-06-23,https://www.nature.com/articles/s43016-022-005...,Open Access,Agricultural Sciences


#### Inspecting a few articles

In [23]:
df_nature_food_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s43016-022-00623-7.pdf'

#### Exporting to a file

In [24]:
df_nature_food_open_access.to_json(f"{output_directory}/nature_food_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Sustainability](https://www.nature.com/natsustain/)

#### Scraping the articles

In [3]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/natsustain/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 37
area_of_knowledge = 'Agricultural Sciences'

In [4]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_sustainability = pd.DataFrame(data)

100%|██████████| 37/37 [00:59<00:00,  1.61s/it]


In [5]:
df_nature_sustainability['Published'] = pd.to_datetime(df_nature_sustainability['Published'])
df_nature_sustainability

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Hybrid polymer network cathode-enabled soluble...,https://www.nature.com/articles/s41893-024-014...,"Meng Liao, Yaobin Xu, Donghai Wang",2024-10-22,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
1,Sustainable lithium extraction and magnesium h...,https://www.nature.com/articles/s41893-024-014...,"Ming Yong, Meng Tang, Huanting Wang",2024-10-22,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
2,Radical climate protests linked to increases i...,https://www.nature.com/articles/s41893-024-014...,"Markus Ostarek, Brent Simpson, James Ozden",2024-10-21,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
3,Involving citizens in monitoring the Kunming–M...,https://www.nature.com/articles/s41893-024-014...,"Finn Danielsen, Natasha Ali, Neil D. Burgess",2024-10-18,https://www.nature.com/articles/s41893-024-014...,Open Access,Agricultural Sciences
4,A renewable and socially accepted energy syste...,https://www.nature.com/articles/s41893-024-014...,"Guillermo Valenzuela-Venegas, Maria Luisa Lode...",2024-10-11,https://www.nature.com/articles/s41893-024-014...,,Agricultural Sciences
...,...,...,...,...,...,...,...
729,Water competition between cities and agricultu...,https://www.nature.com/articles/s41893-017-0006-8,"Martina Flörke, Christof Schneider, Robert I. ...",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences
730,Impacts of booming concrete production on wate...,https://www.nature.com/articles/s41893-017-0009-5,"Sabbie A. Miller, Arpad Horvath, Paulo J. M. M...",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences
731,Targeted emission reductions from global super...,https://www.nature.com/articles/s41893-017-0003-y,"Dan Tong, Qiang Zhang, Kebin He",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences
732,Increased vegetation growth and carbon stock i...,https://www.nature.com/articles/s41893-017-0004-x,"Xiaowei Tong, Martin Brandt, Rasmus Fensholt",2018-01-08,https://www.nature.com/articles/s41893-017-000...,,Agricultural Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [6]:
df_nature_sustainability_open_access = df_nature_sustainability[df_nature_sustainability['Open Access'] == 'Open Access']
df_nature_sustainability_open_access = df_nature_sustainability_open_access[df_nature_sustainability_open_access['Published'] <= '2022-11-30']
df_nature_sustainability_open_access = df_nature_sustainability_open_access.reset_index(drop=True)
df_nature_sustainability_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Towards process-oriented management of tropica...,https://www.nature.com/articles/s41893-022-009...,"Raphael Seguin, David Mouillot, Nicolas Loiseau",2022-11-14,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
1,Impacts of meeting minimum access on critical ...,https://www.nature.com/articles/s41893-022-009...,"Crelis F. Rammelt, Joyeeta Gupta, Caroline Zimm",2022-11-10,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
2,Sustained productivity and agronomic potential...,https://www.nature.com/articles/s41893-022-009...,"Shilai Zhang, Guangfu Huang, Fengyi Hu",2022-11-07,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
3,Protected area personnel and ranger numbers ar...,https://www.nature.com/articles/s41893-022-009...,"Michael R. Appleton, Alexandre Courtiol, Barne...",2022-10-20,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
4,Insect pest management with sex pheromone prec...,https://www.nature.com/articles/s41893-022-009...,"Hong-Lei Wang, Bao-Jian Ding, Christer Löfstedt",2022-09-01,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
5,Quantifying Earth system interactions for sust...,https://www.nature.com/articles/s41893-022-009...,"Anna Chrysafi, Vili Virkki, Matti Kummu",2022-08-15,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
6,Persistent impact of Fukushima decontamination...,https://www.nature.com/articles/s41893-022-009...,"Bin Feng, Yuichi Onda, Yupan Zhang",2022-07-14,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
7,Scientific evidence on the political impact of...,https://www.nature.com/articles/s41893-022-009...,"Frank Biermann, Thomas Hickmann, Birka Wicke",2022-06-20,https://www.nature.com/articles/s41893-022-009...,Open Access,Agricultural Sciences
8,Re-thinking procurement incentives for electri...,https://www.nature.com/articles/s41893-022-008...,"Ashley Nunes, Lucas Woodley, Philip Rossetti",2022-04-04,https://www.nature.com/articles/s41893-022-008...,Open Access,Agricultural Sciences
9,Improving biodiversity protection through arti...,https://www.nature.com/articles/s41893-022-008...,"Daniele Silvestro, Stefano Goria, Alexandre An...",2022-03-24,https://www.nature.com/articles/s41893-022-008...,Open Access,Agricultural Sciences


#### Inspecting a few articles

In [7]:
df_nature_sustainability_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41893-022-00981-x.pdf'

#### Exporting to a file

In [9]:
df_nature_sustainability_open_access.to_json(f"{output_directory}/nature_sustainability_open_access.jsonl", orient='records', lines=True)

### Scraping [Plant Biotechnology Journal](https://onlinelibrary.wiley.com/journal/14677652?msockid=0525cb73d9a76a060b80df20d87e6b4b)

## Applied Social Sciences

### Scraping [Journal of Applied Social Science](https://journals.sagepub.com/home/jax)

#### Capturing a few document pages for inspection

In [46]:
url_sample_1 = 'https://journals.sagepub.com/toc/jaxc/16/2'
url_sample_11 = 'https://journals.sagepub.com/doi/full/10.1177/19367244221078506'

In [47]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/journal_applied_social_science_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/journal_applied_social_science_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [3]:
def scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url}{i}"
        driver.get(url)

        # Explicit wait for main article elements
        wait = WebDriverWait(driver, 10)

        # Ensure stable page load
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)

        # Parse page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # Extract Volume/Issue
        volume_issue_tag = soup.find('title')
        volume_issue = volume_issue_tag.get_text(strip=True) if volume_issue_tag else ''

        # Extract articles
        articles = soup.find_all('div', class_='issue-item issue-item--toc issue-item--checkbox')

        for article in articles:
            # Extracting the article type
            article_type_tag = article.find('span', string='Research article')
            article_type = article_type_tag.get_text(strip=True) if article_type_tag else ''

            # Extracting the title
            title_tag = article.find('div', class_='issue-item__title').find('h5')
            title = title_tag.get_text(strip=True) if title_tag else ''

            # Extracting the article URL
            article_url_tag = article.find('div', class_='issue-item__title').find('a')
            article_url = root_url + article_url_tag['href'] if article_url_tag else ''

            # Extracting authors
            authors_list = article.find('ul', class_='rlist--inline loa comma')
            authors = ', '.join([author.get_text(strip=True) for author in authors_list.find_all('span')]) if authors_list else ''

            # Extracting published date (removing "First published")
            published_tag = article.find('span', string=lambda text: text and text.startswith("First published"))
            published = published_tag.get_text(strip=True).replace("First published ", "") if published_tag else ''

            # Extracting DOI
            doi_tag = article.find('input', {'name': 'doi'})
            doi = doi_root_url + doi_tag['value'] if doi_tag else ''

            # Extracting Open Access status
            open_access_tag = article.find('span', class_='issue-item-access').find('span')
            open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

            # Extracting PDF URL (Replacing 'reader' with 'pdf')
            pdf_url_tag = article.find('a', title='download')
            pdf_url = root_url + pdf_url_tag['href'].replace('reader', 'pdf') if pdf_url_tag and pdf_url_tag.get('href') else ''

            # Append extracted data
            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': published,
                'DOI': doi,
                'Open Access': open_access,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 14 (Issue 1, March 2020; Issue 2, September 2020)

In [8]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/jaxc/14/'
start_page = 1
end_page = 2
area_of_knowledge = 'Applied Social Sciences'

In [9]:
df_v14i1i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:18<00:00,  9.48s/it]


##### Volume 15 (Issue 1, March 2021; Issue 2, September 2021)

In [10]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/jaxc/15/'
start_page = 1
end_page = 2
area_of_knowledge = 'Applied Social Sciences'

In [11]:
df_v15i1i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:19<00:00,  9.56s/it]


##### Volume 16 (Issue 1, March 2022; Issue 2, June 2022; Issue 3, September 2022)

In [12]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/jaxc/16/'
start_page = 1
end_page = 3
area_of_knowledge = 'Applied Social Sciences'

In [13]:
df_v16i1i2i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 3/3 [00:23<00:00,  7.70s/it]


##### Volume 17 (Issue 1, March 2023)

In [14]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/jaxc/17/'
start_page = 1
end_page = 1
area_of_knowledge = 'Applied Social Sciences'

In [15]:
df_v17i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.78s/it]


##### Concatenating the DataFrames

In [16]:
df_journal_applied_social_science = pd.concat([df_v14i1i2, df_v15i1i2, df_v16i1i2i3, df_v17i1], ignore_index=True)

In [19]:
df_journal_applied_social_science['Published'] = pd.to_datetime(
    df_journal_applied_social_science['Published'], format='%B %d, %Y'
)

In [20]:
df_journal_applied_social_science.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [21]:
unique_open_access = df_journal_applied_social_science['Open Access'].dropna().unique()
unique_open_access

array(['Free access', 'Restricted access', 'Open Access'], dtype=object)

In [23]:
df_journal_applied_social_science_open_access = df_journal_applied_social_science[
    (df_journal_applied_social_science['Open Access'] == 'Open Access') | 
    (df_journal_applied_social_science['Open Access'] == 'Free access')
]
df_journal_applied_social_science_open_access = df_journal_applied_social_science_open_access[df_journal_applied_social_science_open_access['Published'] <= '2022-11-30']
df_journal_applied_social_science_open_access = df_journal_applied_social_science_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [24]:
df_journal_applied_social_science_open_access = df_journal_applied_social_science_open_access.loc[
    df_journal_applied_social_science_open_access['Article Type'] == 'Research article'
].reset_index(drop=True)
df_journal_applied_social_science_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,PDF URL,Area of Knowledge
0,Research article,“People Have to Comply with the Measures”: Cov...,https://journals.sagepub.com/doi/abs/10.1177/1...,Costas S. Constantinou,"Journal of Applied Social Science - Volume 15,...",2020-12-17,https://doi.org/10.1177/1936724420980374,Free access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
1,Research article,An Essay on the Challenges of Doing Education ...,https://journals.sagepub.com/doi/abs/10.1177/1...,Karen Robson,"Journal of Applied Social Science - Volume 15,...",2021-06-10,https://doi.org/10.1177/19367244211003471,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
2,Research article,Sociotherapy in the Time of COVID-19: A Critic...,https://journals.sagepub.com/doi/abs/10.1177/1...,Sheila L. Cavanagh,"Journal of Applied Social Science - Volume 15,...",2021-06-01,https://doi.org/10.1177/1936724421998275,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
3,Research article,Using Institutional Ethnography to Bridge the ...,https://journals.sagepub.com/doi/abs/10.1177/1...,Kristin Atwood,"Journal of Applied Social Science - Volume 15,...",2021-04-22,https://doi.org/10.1177/19367244211000709,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
4,Research article,Reflections on Embeddedness in an Applied Soci...,https://journals.sagepub.com/doi/abs/10.1177/1...,"Grace Maich, Jeff Boggs, Jonah Butovsky","Journal of Applied Social Science - Volume 15,...",2021-03-17,https://doi.org/10.1177/19367244211000271,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
5,Research article,"Staying Home, Staying Alive: Campus Food Pantr...",https://journals.sagepub.com/doi/abs/10.1177/1...,Elmira Jangjou,"Journal of Applied Social Science - Volume 16,...",2021-08-05,https://doi.org/10.1177/19367244211035671,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
6,Research article,Social Security Policy in Response to the Pand...,https://journals.sagepub.com/doi/abs/10.1177/1...,Le Hoang Viet Lam,"Journal of Applied Social Science - Volume 16,...",2021-07-08,https://doi.org/10.1177/19367244211026912,Free access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
7,Research article,Improving Data Quality with Sociodemographic M...,https://journals.sagepub.com/doi/abs/10.1177/1...,Felix Bittmann,"Journal of Applied Social Science - Volume 16,...",2021-03-19,https://doi.org/10.1177/1936724421998257,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
8,Research article,Determinants of COVID-19 Response Strategies i...,https://journals.sagepub.com/doi/abs/10.1177/1...,"Peter Asare-Nuamah, Christopher Dick-Sagoe, [....","Journal of Applied Social Science - Volume 16,...",2021-12-14,https://doi.org/10.1177/19367244211055799,Free access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences
9,Research article,Modeling the Social Determinants of Official C...,https://journals.sagepub.com/doi/abs/10.1177/1...,Marcell T. Kurbucz,"Journal of Applied Social Science - Volume 16,...",2021-12-17,https://doi.org/10.1177/19367244211061671,Free access,https://journals.sagepub.com/doi/pdf/10.1177/1...,Applied Social Sciences


#### Inspecting a few articles

In [25]:
df_journal_applied_social_science_open_access.at[0, 'URL']

'https://journals.sagepub.com/doi/abs/10.1177/1936724420980374'

#### Exporting to a file

In [26]:
df_journal_applied_social_science_open_access.to_json(f"{output_directory}/journal_applied_social_science_open_access.jsonl", orient='records', lines=True)

### Scraping [Journal of Social Issues](https://spssi.onlinelibrary.wiley.com/journal/15404560)

#### Capturing a few document pages for inspection

Wiley's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [44]:
url_sample_1 = 'https://spssi.onlinelibrary.wiley.com/toc/15404560/2022/78/4'
url_sample_11 = 'https://spssi.onlinelibrary.wiley.com/doi/10.1111/josi.12561'

In [45]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/journal_social_issues_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Closing the WebDriver
driver.quit()

In [45]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/journal_social_issues_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [23]:
def scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url}{i}"
        driver.get(url)

        # Explicit wait for main article elements
        wait = WebDriverWait(driver, 10)

        # Ensure stable page load
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)

        # Parse page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # Extract Volume/Issue
        volume_issue = f"{volume}, Issue {i}"

        # Extract articles
        articles = soup.find_all('div', class_='issue-item')
        
        for article in articles:
            # Extract article type (if applicable)
            article_type = "Research article"  # Default assumption if not explicitly listed
        
            # Extract title
            title_tag = article.find('a', class_='issue-item__title').find('h2')
            title = title_tag.get_text(strip=True) if title_tag else ''
        
            # Extract article URL
            article_url_tag = article.find('a', class_='issue-item__title')
            article_url = root_url + article_url_tag['href'] if article_url_tag else ''
        
            # Extract authors
            authors_list = article.find('div', class_='comma__list')
            authors = ', '.join([author.get_text(strip=True) for author in authors_list.find_all('span', class_='author-style')]) if authors_list else ''
        
            # Extract published date (removing "First Published: ")
            published_tag = article.find('li', class_='ePubDate')
            published = published_tag.find_all('span')[1].get_text(strip=True) if published_tag else ''
        
            # Extract DOI
            doi_tag = article.find('input', {'type': 'hidden'})
            doi = doi_root_url + doi_tag['value'].replace('%2F', '/') if doi_tag else ''
        
            # Extract Open Access status
            open_access_tag = article.find('div', class_='doi-access')
            open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''
        
            # Extract PDF URL
            pdf_url_tag = article.find('li', class_='PdfLink').find('a')
            pdf_url = pdf_url_tag['href'].replace('epdf', 'pdf') if pdf_url_tag and pdf_url_tag.get('href') else ''
        
            # Append extracted data
            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': published,
                'DOI': doi,
                'Open Access': open_access,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 76 (Issue 1; Issue 2; Issue 3; Issue 4)

Wiley's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [24]:
root_url = 'https://spssi.onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://spssi.onlinelibrary.wiley.com/toc/15404560/2020/76/'
volume = 'Volume 76'
area_of_knowledge = 'Applied Social Sciences'

In [25]:
start_page = 1
end_page = 1

In [26]:
df_v76i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:21<00:00, 21.67s/it]


In [28]:
start_page = 2
end_page = 2

In [29]:
df_v76i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.39s/it]


In [30]:
start_page = 3
end_page = 3

In [31]:
df_v76i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.47s/it]


In [32]:
start_page = 4
end_page = 4

In [33]:
df_v76i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.01s/it]


##### Volume 77 (Issue 1; Issue 2; Issue 3; Issue 4)

In [37]:
root_url = 'https://spssi.onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://spssi.onlinelibrary.wiley.com/toc/15404560/2021/77/'
volume = 'Volume 77'
area_of_knowledge = 'Applied Social Sciences'

In [38]:
start_page = 1
end_page = 1

In [39]:
df_v77i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.41s/it]


In [40]:
start_page = 2
end_page = 2

In [41]:
df_v77i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.30s/it]


In [42]:
start_page = 3
end_page = 3

In [43]:
df_v77i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.93s/it]


In [44]:
start_page = 4
end_page = 4

In [45]:
df_v77i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.68s/it]


##### Volume 78 (Issue 1; Issue 2; Issue 3; Issue 4)

In [46]:
root_url = 'https://spssi.onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://spssi.onlinelibrary.wiley.com/toc/15404560/2022/78/'
volume = 'Volume 78'
area_of_knowledge = 'Applied Social Sciences'

In [47]:
start_page = 1
end_page = 1

In [48]:
df_v78i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.11s/it]


In [49]:
start_page = 2
end_page = 2

In [50]:
df_v78i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.88s/it]


In [51]:
start_page = 3
end_page = 3

In [52]:
df_v78i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:14<00:00, 14.66s/it]


In [53]:
start_page = 4
end_page = 4

In [54]:
df_v78i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.21s/it]


##### Concatenating the DataFrames

In [55]:
df_journal_social_issues = pd.concat([
    df_v76i1, df_v76i2, df_v76i3, df_v76i4,
    df_v77i1, df_v77i2, df_v77i3, df_v77i4,
    df_v78i1, df_v78i2, df_v78i3, df_v78i4
], ignore_index=True)

In [58]:
df_journal_social_issues['Published'] = pd.to_datetime(
    df_journal_social_issues['Published'], format='mixed', errors='coerce'
)

In [59]:
df_journal_social_issues.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [61]:
unique_open_access = df_journal_social_issues['Open Access'].dropna().unique()
unique_open_access

array(['Free Access', '', 'Open Access', 'Free to Read'], dtype=object)

In [62]:
df_journal_social_issues_open_access = df_journal_social_issues[
    (df_journal_social_issues['Open Access'] == 'Open Access') | 
    (df_journal_social_issues['Open Access'] == 'Free Access')
]
df_journal_social_issues_open_access = df_journal_social_issues_open_access[df_journal_social_issues_open_access['Published'] <= '2022-11-30']
df_journal_social_issues_open_access = df_journal_social_issues_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [64]:
df_journal_social_issues_open_access = df_journal_social_issues_open_access.loc[
    ~df_journal_social_issues_open_access['Title'].isin(['Issue Information', 'Erratum'])
].reset_index(drop=True)

In [65]:
df_journal_social_issues_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,PDF URL,Area of Knowledge
0,Research article,Sustainable Consumption: The Psychology of Ind...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Matthew B Ruby, Iain Walker, Hanne M Watkins","Volume 76, Issue 1",2020-03-05,https://doi.org/10.1111/josi.12376,Free Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
1,Research article,My Influence on the World (of Others): Goal Ef...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Karen R. S. Hamann, Gerhard Reese","Volume 76, Issue 1",2020-01-29,https://doi.org/10.1111/josi.12369,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
2,Research article,The Perils of Explaining Climate Inaction in T...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Michael T. Schmitt, Scott D. Neufeld, Caroline...","Volume 76, Issue 1",2019-11-19,https://doi.org/10.1111/josi.12360,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
3,Research article,How do we get people into contact? Predictors ...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Mathias Kauff, Marta Beneda, Stefania Paolini,...","Volume 77, Issue 1",2020-10-31,https://doi.org/10.1111/josi.12398,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
4,Research article,Intimate intergroup contact across the lifespan,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Marco Marinucci, Rachel Maunder, Kiara Sanchez...","Volume 77, Issue 1",2020-10-19,https://doi.org/10.1111/josi.12399,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
5,Research article,Does negative contact undermine attempts to im...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Sarina J. Schäfer, Mathias Kauff, Francesca Pr...","Volume 77, Issue 1",2021-03-02,https://doi.org/10.1111/josi.12422,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
6,Research article,"Negative contact, collective action, and socia...",https://spssi.onlinelibrary.wiley.com/doi/10.1...,"John Dixon, Shelley McKeown","Volume 77, Issue 1",2021-03-10,https://doi.org/10.1111/josi.12429,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
7,Research article,Self-affirmation theory in educational contexts,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Matthew J. Easterbrook, Peter R. Harris, David...","Volume 77, Issue 3",2021-06-29,https://doi.org/10.1111/josi.12459,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
8,Research article,Dual identity in context: The role of minority...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Judit Kende, Gülseli Baysu, Karen Phalet, Fene...","Volume 77, Issue 4",2021-12-11,https://doi.org/10.1111/josi.12487,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences
9,Research article,Adolescents’ own and parental expectations for...,https://spssi.onlinelibrary.wiley.com/doi/10.1...,"Jeanine Grütter, Sandesh Dhakal, Melanie Killen","Volume 77, Issue 4",2021-12-11,https://doi.org/10.1111/josi.12484,Open Access,https://spssi.onlinelibrary.wiley.com/doi/pdf/...,Applied Social Sciences


#### Inspecting a few articles

**Attention**: Some of the documents might in fact be `Editorials` rather than `Research articles`. Extra refinement is recommended when using the dataset.

In [66]:
df_journal_social_issues_open_access.at[0, 'URL']

'https://spssi.onlinelibrary.wiley.com/doi/10.1111/josi.12376'

#### Exporting to a file

In [67]:
df_journal_social_issues_open_access.to_json(f"{output_directory}/journal_social_issues_open_access.jsonl", orient='records', lines=True)

### Scraping [Social Science & Medicine](https://www.sciencedirect.com/journal/social-science-and-medicine)

#### Capturing a few document pages for inspection

In [8]:
url_sample_1 = 'https://www.sciencedirect.com/journal/social-science-and-medicine/vol/244/suppl/C'
#url_sample_2 = 'https://www.sciencedirect.com/journal/social-science-and-medicine/vol/313/suppl/C'
url_sample_11 = 'https://www.sciencedirect.com/science/article/pii/S0277953619305933'

In [9]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
#driver.get(url_sample_2)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/social_science_medicine_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

#document_page_sample_2 = driver.page_source

#with open(f'{output_directory}/social_science_medicine_sample2.html', 'w', encoding='utf8', newline='\n') as file:
#    file.write(document_page_sample_2)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/social_science_medicine_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [21]:
def scrape_articles(article_list_url1, article_list_url2, root_url, start_page, end_page, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url1}{i}{article_list_url2}"
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        #wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')

        volume_issue_tag = soup.find('h2', class_='u-text-light js-vol-issue')
        volume_issue = volume_issue_tag.get_text(strip=True) if volume_issue_tag else ''

        month_year_tag = soup.find('h3', class_='js-issue-status text-s')
        month_year = month_year_tag.get_text(strip=True) if month_year_tag else ''

        articles = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')

        for article in articles:
            article_type_tag = article.find('span', class_='js-article-subtype')
            article_type = article_type_tag.get_text(strip=True) if article_type_tag else ''

            title_tag = article.find('span', class_='js-article-title text-l')
            title = title_tag.get_text(strip=True) if title_tag else ''

            article_url_tag = article.find('h3', class_='text-m u-font-serif u-display-inline').find('a')
            article_url = root_url + article_url_tag['href'] if article_url_tag else ''

            authors_tags = article.find('div', class_='text-s u-clr-grey8 js-article__item__authors')
            authors = authors_tags.get_text(strip=True) if authors_tags else ''

            doi_tag = article.find('div', attrs={'hidden': True})  # Explicitly target the 'hidden' attribute
            doi = doi_tag.get_text(strip=True) if doi_tag else ''

            open_access_tag = article.find('span', class_='js-open-access u-text-italic')
            open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

            open_access1_tag = article.find('span', class_='js-open-access')
            open_access1 = open_access1_tag.get_text(strip=True) if open_access1_tag else ''

            pdf_url_tag = article.find('dd', class_='js-article-link u-margin-xs-top u-margin-s-bottom article-item-link')
            pdf_url = root_url + pdf_url_tag.find('a')['href'] if pdf_url_tag and pdf_url_tag.find('a') else ''

            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': month_year,
                'DOI': doi,
                'Open Access': open_access,
                'Open Access 1': open_access1,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    driver.quit()

    return pd.DataFrame(data)

##### Volumes 244 to 313

In [22]:
root_url = 'https://www.sciencedirect.com/'
article_list_url1 = 'https://www.sciencedirect.com/journal/social-science-and-medicine/vol/'
article_list_url2 = '/suppl/C'
start_page = 244
end_page = 313
area_of_knowledge = 'Applied Social Sciences'

In [23]:
df_social_science_medicine = scrape_articles(article_list_url1, article_list_url2, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 70/70 [07:00<00:00,  6.00s/it]


In [24]:
df_social_science_medicine['Published'] = pd.to_datetime(
    df_social_science_medicine['Published'], format='%B %Y'
)

In [25]:
df_social_science_medicine.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
Open Access 1                object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [26]:
unique_open_access = df_social_science_medicine['Open Access'].dropna().unique()
unique_open_access

array(['', 'Open access'], dtype=object)

In [27]:
unique_open_access1 = df_social_science_medicine['Open Access 1'].dropna().unique()
unique_open_access1

array(['Full text access', 'Abstract only', 'Open access', 'No access'],
      dtype=object)

In [28]:
df_social_science_medicine_open_access = df_social_science_medicine[
    (df_social_science_medicine['Open Access'] == 'Open access') | 
    (df_social_science_medicine['Open Access 1'] == 'Full text access') | 
    (df_social_science_medicine['Open Access 1'] == 'Open access')
]
df_social_science_medicine_open_access = df_social_science_medicine_open_access[df_social_science_medicine_open_access['Published'] <= '2022-11-30']
df_social_science_medicine_open_access = df_social_science_medicine_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [29]:
df_social_science_medicine_open_access = df_social_science_medicine_open_access.loc[
    df_social_science_medicine_open_access['Article Type'] == 'Research article'
].reset_index(drop=True)
df_social_science_medicine_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,Open Access 1,PDF URL,Area of Knowledge
0,Research article,Making a murderer: Media renderings of brain i...,https://www.sciencedirect.com//science/article...,Hollin Gregory,Volume 244,2020-01-01,https://doi.org/10.1016/j.socscimed.2019.112598,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
1,Research article,Defining and measuring health poverty,https://www.sciencedirect.com//science/article...,"Philip Clarke, Guido Erreygers",Volume 244,2020-01-01,https://doi.org/10.1016/j.socscimed.2019.112633,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
2,Research article,A critical content analysis of media reporting...,https://www.sciencedirect.com//science/article...,"Fiona Webster, Kathleen Rice, Abhimanyu Sud",Volume 244,2020-01-01,https://doi.org/10.1016/j.socscimed.2019.112642,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
3,Research article,"“It was all taken away”: Lactation, embodiment...",https://www.sciencedirect.com//science/article...,"Aunchalee E.L. Palmquist, Sarah M. Holdren, Cy...",Volume 244,2020-01-01,https://doi.org/10.1016/j.socscimed.2019.112648,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
4,Research article,"Care providers, access to care, and the Long-t...",https://www.sciencedirect.com//science/article...,"Shuang Chang, Wei Yang, Hiroshi Deguchi",Volume 244,2020-01-01,https://doi.org/10.1016/j.socscimed.2019.112667,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
...,...,...,...,...,...,...,...,...,...,...,...
720,Research article,The health cost of reducing hospital bed capacity,https://www.sciencedirect.com//science/article...,"Jonathan Siverskog, Martin Henriksson",Volume 313,2022-11-01,https://doi.org/10.1016/j.socscimed.2022.115399,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
721,Research article,"Water journeys: Household water insecurity, he...",https://www.sciencedirect.com//science/article...,"Ellis Adjei Adams, Sydney Byrns, Save Kumwenda...",Volume 313,2022-11-01,https://doi.org/10.1016/j.socscimed.2022.115394,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
722,Research article,Theoretical mediators of GlobalConsent: An ada...,https://www.sciencedirect.com//science/article...,"Kathryn M. Yount, Irina Bergenfeld, Katherine ...",Volume 313,2022-11-01,https://doi.org/10.1016/j.socscimed.2022.115402,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences
723,Research article,The ubiquity of uncertainty in low back pain care,https://www.sciencedirect.com//science/article...,"Nathalia Costa, Karime Mescouto, Miriam Dillon...",Volume 313,2022-11-01,https://doi.org/10.1016/j.socscimed.2022.115422,Open access,Open access,https://www.sciencedirect.com//science/article...,Applied Social Sciences


#### Inspecting a few articles

In [30]:
df_social_science_medicine_open_access.at[0, 'URL']

'https://www.sciencedirect.com//science/article/pii/S0277953619305933'

#### Exporting to a file

In [31]:
df_social_science_medicine_open_access.to_json(f"{output_directory}/social_science_medicine_open_access.jsonl", orient='records', lines=True)

### Scraping [Sociological Practice Association](https://digitalcommons.wayne.edu/socprac/)

This journal has been excluded from the study because it is too old.

## Biological Sciences

### Scraping [Cell](https://www.cell.com/cell/home)

#### Capturing a few document pages for inspection

In [3]:
url_sample_1 = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0002-1'
url_sample_11 = 'https://www.cell.com/cell/fulltext/S0092-8674(19)31270-X'

In [4]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/cell_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Closing the WebDriver
driver.quit()

In [5]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/cell_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [3]:
def scrape_articles(root_url, article_list_url, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    driver.get(article_list_url)

    # Explicit wait for main article elements
    wait = WebDriverWait(driver, 10)

    # Ensure stable page load
    max_wait_time = 30
    start_time = time.time()
    previous_html = ''
    while True:
        current_html = driver.page_source
        if current_html == previous_html or time.time() - start_time > max_wait_time:
            break
        previous_html = current_html
        time.sleep(2)

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # Extract Published/Volume/Issue
    published_tag = soup.find('h1', class_='toc-header__issue-date')
    published = published_tag.get_text(strip=True) if published_tag else ''

    volume_tag = soup.find('span', class_='toc-header__volume')
    volume = volume_tag.get_text(strip=True).replace('Volume ', '') if volume_tag else ''

    issue_tag = soup.find('span', class_='toc-header__issue')
    issue = issue_tag.get_text(strip=True).replace('Issue ', '') if issue_tag else ''

    # Extract articles
    articles = []
    articles_header = soup.find('h2', string='Articles')
    if articles_header:
        articles_list = articles_header.find_next('ul', class_='toc__body')
        if articles_list:
            articles = articles_list.find_all('li', class_='articleCitation')

    for article in articles:
        # Extract article type
        article_type = 'Article'
    
        # Extract title
        title_tag = article.find('h3', class_='toc__item__title')
        title = title_tag.get_text(strip=True) if title_tag else ''
    
        # Extract article URL
        article_url_tag = title_tag.find('a') if title_tag else None
        article_url = f"{root_url}{article_url_tag.get('href', '')}" if article_url_tag else ''
    
        # Extract authors
        authors_list = article.find('ul', class_='toc__item__authors')
        authors = ', '.join([author.get_text(strip=True).rstrip(',') for author in authors_list.find_all('li', class_='loa__item')]) if authors_list else ''
    
        # Extract PDF URL
        pdf_link_tag = article.find('a', class_='pdfLink')
        pdf_url = f"{root_url}{pdf_link_tag.get('href', '')}" if pdf_link_tag else ''
    
        # Append extracted data
        data.append({
            'Article Type': article_type,
            'Title': title,
            'URL': article_url,
            'Authors': authors,
            'Vol/Issue': f"Volume {volume}/Issue {issue}",
            'Published': published,
            'PDF URL': pdf_url,
            'Area of Knowledge': area_of_knowledge
        })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 180 (Issue 1 - 6)

Cell's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [4]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0002-1'
area_of_knowledge = 'Biological Sciences'

In [5]:
df_v180i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [6]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0003-3'
area_of_knowledge = 'Biological Sciences'

In [7]:
df_v180i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [8]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0004-5'
area_of_knowledge = 'Biological Sciences'

In [9]:
df_v180i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [10]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0005-7'
area_of_knowledge = 'Biological Sciences'

In [11]:
df_v180i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [12]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0006-9'
area_of_knowledge = 'Biological Sciences'

In [13]:
df_v180i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [14]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0007-0'
area_of_knowledge = 'Biological Sciences'

In [15]:
df_v180i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 181 (Issue 1 - 7)

In [16]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0008-2'
area_of_knowledge = 'Biological Sciences'

In [17]:
df_v181i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [18]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0009-4'
area_of_knowledge = 'Biological Sciences'

In [19]:
df_v181i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [20]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0010-0'
area_of_knowledge = 'Biological Sciences'

In [21]:
df_v181i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [22]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0011-2'
area_of_knowledge = 'Biological Sciences'

In [23]:
df_v181i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [24]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0012-4'
area_of_knowledge = 'Biological Sciences'

In [25]:
df_v181i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [26]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0013-6'
area_of_knowledge = 'Biological Sciences'

In [27]:
df_v181i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [28]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0014-8'
area_of_knowledge = 'Biological Sciences'

In [29]:
df_v181i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 182 (Issue 1 - 6)

In [30]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0015-X'
area_of_knowledge = 'Biological Sciences'

In [31]:
df_v182i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [32]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0016-1'
area_of_knowledge = 'Biological Sciences'

In [33]:
df_v182i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [34]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0017-3'
area_of_knowledge = 'Biological Sciences'

In [35]:
df_v182i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [36]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0018-5'
area_of_knowledge = 'Biological Sciences'

In [37]:
df_v182i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [38]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0019-7'
area_of_knowledge = 'Biological Sciences'

In [39]:
df_v182i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [40]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0020-3'
area_of_knowledge = 'Biological Sciences'

In [41]:
df_v182i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 183 (Issue 1 - 7)

In [42]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0021-5'
area_of_knowledge = 'Biological Sciences'

In [43]:
df_v183i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [44]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0022-7'
area_of_knowledge = 'Biological Sciences'

In [45]:
df_v183i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [46]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0023-9'
area_of_knowledge = 'Biological Sciences'

In [47]:
df_v183i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [48]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0024-0'
area_of_knowledge = 'Biological Sciences'

In [49]:
df_v183i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [50]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0025-2'
area_of_knowledge = 'Biological Sciences'

In [51]:
df_v183i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [52]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0026-4'
area_of_knowledge = 'Biological Sciences'

In [53]:
df_v183i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [54]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(19)X0027-6'
area_of_knowledge = 'Biological Sciences'

In [55]:
df_v183i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 184 (Issue 1 - 26)

In [56]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0002-X'
area_of_knowledge = 'Biological Sciences'

In [57]:
df_v184i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [58]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0003-1'
area_of_knowledge = 'Biological Sciences'

In [59]:
df_v184i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [60]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0004-3'
area_of_knowledge = 'Biological Sciences'

In [61]:
df_v184i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [62]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0005-5'
area_of_knowledge = 'Biological Sciences'

In [63]:
df_v184i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [64]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0006-7'
area_of_knowledge = 'Biological Sciences'

In [65]:
df_v184i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [66]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0007-9'
area_of_knowledge = 'Biological Sciences'

In [67]:
df_v184i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [68]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0008-0'
area_of_knowledge = 'Biological Sciences'

In [69]:
df_v184i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [70]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0009-2'
area_of_knowledge = 'Biological Sciences'

In [71]:
df_v184i8 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [72]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0010-9'
area_of_knowledge = 'Biological Sciences'

In [73]:
df_v184i9 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [74]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0011-0'
area_of_knowledge = 'Biological Sciences'

In [75]:
df_v184i10 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [76]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0012-2'
area_of_knowledge = 'Biological Sciences'

In [77]:
df_v184i11 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [78]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0013-4'
area_of_knowledge = 'Biological Sciences'

In [79]:
df_v184i12 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [80]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0014-6'
area_of_knowledge = 'Biological Sciences'

In [81]:
df_v184i13 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [82]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0015-8'
area_of_knowledge = 'Biological Sciences'

In [83]:
df_v184i14 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [84]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0016-X'
area_of_knowledge = 'Biological Sciences'

In [85]:
df_v184i15 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [86]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0017-1'
area_of_knowledge = 'Biological Sciences'

In [87]:
df_v184i16 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [88]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0018-3'
area_of_knowledge = 'Biological Sciences'

In [89]:
df_v184i17 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [90]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0019-5'
area_of_knowledge = 'Biological Sciences'

In [91]:
df_v184i18 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [92]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0020-1'
area_of_knowledge = 'Biological Sciences'

In [93]:
df_v184i19 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [94]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0021-3'
area_of_knowledge = 'Biological Sciences'

In [95]:
df_v184i20 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [96]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0022-5'
area_of_knowledge = 'Biological Sciences'

In [97]:
df_v184i21 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [98]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0023-7'
area_of_knowledge = 'Biological Sciences'

In [99]:
df_v184i22 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [100]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0024-9'
area_of_knowledge = 'Biological Sciences'

In [101]:
df_v184i23 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [102]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0025-0'
area_of_knowledge = 'Biological Sciences'

In [103]:
df_v184i24 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [104]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0026-2'
area_of_knowledge = 'Biological Sciences'

In [105]:
df_v184i25 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [106]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(20)X0027-4'
area_of_knowledge = 'Biological Sciences'

In [107]:
df_v184i26 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 185 (Issue 1 - 24)

In [108]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0002-5'
area_of_knowledge = 'Biological Sciences'

In [109]:
df_v185i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [110]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0003-7'
area_of_knowledge = 'Biological Sciences'

In [111]:
df_v185i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [112]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0004-9'
area_of_knowledge = 'Biological Sciences'

In [113]:
df_v185i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [114]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0005-0'
area_of_knowledge = 'Biological Sciences'

In [115]:
df_v185i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [116]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0006-2'
area_of_knowledge = 'Biological Sciences'

In [117]:
df_v185i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [118]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0007-4'
area_of_knowledge = 'Biological Sciences'

In [119]:
df_v185i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [120]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0008-6'
area_of_knowledge = 'Biological Sciences'

In [121]:
df_v185i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [122]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0009-8'
area_of_knowledge = 'Biological Sciences'

In [123]:
df_v185i8 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [124]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0010-4'
area_of_knowledge = 'Biological Sciences'

In [125]:
df_v185i9 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [126]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0011-6'
area_of_knowledge = 'Biological Sciences'

In [127]:
df_v185i10 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [128]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0012-8'
area_of_knowledge = 'Biological Sciences'

In [129]:
df_v185i11 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [130]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0013-X'
area_of_knowledge = 'Biological Sciences'

In [131]:
df_v185i12 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [132]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0014-1'
area_of_knowledge = 'Biological Sciences'

In [133]:
df_v185i13 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [134]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0015-3'
area_of_knowledge = 'Biological Sciences'

In [135]:
df_v185i14 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [136]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0016-5'
area_of_knowledge = 'Biological Sciences'

In [137]:
df_v185i15 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [138]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0017-7'
area_of_knowledge = 'Biological Sciences'

In [139]:
df_v185i16 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [140]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0018-9'
area_of_knowledge = 'Biological Sciences'

In [141]:
df_v185i17 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [142]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0019-0'
area_of_knowledge = 'Biological Sciences'

In [143]:
df_v185i18 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [144]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0020-7'
area_of_knowledge = 'Biological Sciences'

In [145]:
df_v185i19 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [146]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0021-9'
area_of_knowledge = 'Biological Sciences'

In [147]:
df_v185i20 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [148]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0022-0'
area_of_knowledge = 'Biological Sciences'

In [149]:
df_v185i21 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [150]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0023-2'
area_of_knowledge = 'Biological Sciences'

In [151]:
df_v185i22 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [152]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0024-4'
area_of_knowledge = 'Biological Sciences'

In [153]:
df_v185i23 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [154]:
root_url = 'https://www.cell.com'
article_list_url = 'https://www.cell.com/cell/issue?pii=S0092-8674(21)X0025-6'
area_of_knowledge = 'Biological Sciences'

In [155]:
df_v185i24 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Concatenating the DataFrames

In [156]:
df_cell = pd.concat([
    df_v180i1, df_v180i2, df_v180i3, df_v180i4, df_v180i5, df_v180i6,
    df_v181i1, df_v181i2, df_v181i3, df_v181i4, df_v181i5, df_v181i6, df_v181i7,
    df_v182i1, df_v182i2, df_v182i3, df_v182i4, df_v182i5, df_v182i6,
    df_v183i1, df_v183i2, df_v183i3, df_v183i4, df_v183i5, df_v183i6, df_v183i7,
    df_v184i1, df_v184i2, df_v184i3, df_v184i4, df_v184i5, df_v184i6, df_v184i7, df_v184i8, df_v184i9, df_v184i10, df_v184i11, df_v184i12, df_v184i13, df_v184i14, df_v184i15, df_v184i16, df_v184i17, df_v184i18, df_v184i19, df_v184i20, df_v184i21, df_v184i22, df_v184i23, df_v184i24, df_v184i25, df_v184i26,
    df_v185i1, df_v185i2, df_v185i3, df_v185i4, df_v185i5, df_v185i6, df_v185i7, df_v185i8, df_v185i9, df_v185i10, df_v185i11, df_v185i12, df_v185i13, df_v185i14, df_v185i15, df_v185i16, df_v185i17, df_v185i18, df_v185i19, df_v185i20, df_v185i21, df_v185i22, df_v185i23, df_v185i24
], ignore_index=True)

In [159]:
df_cell['Published'] = pd.to_datetime(
    df_cell['Published'], format='mixed', errors='coerce'
)

In [162]:
df_cell.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [163]:
df_cell_open_access = df_cell
df_cell_open_access = df_cell_open_access[df_cell_open_access['Published'] <= '2022-11-30']
df_cell_open_access = df_cell_open_access.reset_index(drop=True)

In [164]:
df_cell_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,PDF URL,Area of Knowledge
0,Article,Gut-Innervating Nociceptor Neurons Regulate Pe...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Nicole Y. Lai, Melissa A. Musser, Felipe A. Pi...",Volume 180/Issue 1,2020-01-09,https://www.cell.com/cell/pdf/S0092-8674(19)31...,Biological Sciences
1,Article,Enteric Nervous System-Derived IL-18 Orchestra...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Abigail Jarret, Ruaidhrí Jackson, Coco Duizer,...",Volume 180/Issue 1,2020-01-09,https://www.cell.com/cell/pdf/S0092-8674(19)31...,Biological Sciences
2,Article,Adrenergic Signaling in Muscularis Macrophages...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Fanny Matheis, Paul A. Muller, Christina L. Gr...",Volume 180/Issue 1,2020-01-09,https://www.cell.com/cell/pdf/S0092-8674(19)31...,Biological Sciences
3,Article,Serum Amyloid A Proteins Induce Pathogenic Th1...,https://www.cell.com/cell/fulltext/S0092-8674(...,"June-Yong Lee, Jason A. Hall, Lina Kroehling, ...",Volume 180/Issue 1,2020-01-09,https://www.cell.com/cell/pdf/S0092-8674(19)31...,Biological Sciences
4,Article,Restricted Clonality and Limited Germinal Cent...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Luka Mesin, Ariën Schiepers, Jonatan Ersching,...",Volume 180/Issue 1,2020-01-09,https://www.cell.com/cell/pdf/S0092-8674(19)31...,Biological Sciences
...,...,...,...,...,...,...,...,...
708,Article,A microbial transporter of the dietary antioxi...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Daniel G. Dumitrescu, Elizabeth M. Gordon, Yek...",Volume 185/Issue 24,2022-11-23,https://www.cell.com/cell/pdf/S0092-8674(22)01...,Biological Sciences
709,Article,Mechanoreceptor signal convergence and transfo...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Anda M. Chirila, Genelle Rankin, Shih-Yi Tseng...",Volume 185/Issue 24,2022-11-23,https://www.cell.com/cell/pdf/S0092-8674(22)01...,Biological Sciences
710,Article,Membrane phosphoinositides regulate GPCR-β-arr...,https://www.cell.com/cell/fulltext/S0092-8674(...,"John Janetzko, Ryoji Kise, Benjamin Barsi-Rhyn...",Volume 185/Issue 24,2022-11-23,https://www.cell.com/cell/pdf/S0092-8674(22)01...,Biological Sciences
711,Article,Diverse virus-encoded CRISPR-Cas systems inclu...,https://www.cell.com/cell/fulltext/S0092-8674(...,"Basem Al-Shayeb, Petr Skopintsev, Katarzyna M....",Volume 185/Issue 24,2022-11-23,https://www.cell.com/cell/pdf/S0092-8674(22)01...,Biological Sciences


#### Inspecting a few articles

In [165]:
df_cell_open_access.at[4, 'URL']

'https://www.cell.com/cell/fulltext/S0092-8674(19)31317-0'

#### Exporting to a file

In [166]:
df_cell_open_access.to_json(f"{output_directory}/cell_open_access.jsonl", orient='records', lines=True)

### Scraping [American Journal of Human Biology](https://onlinelibrary.wiley.com/journal/15206300?msockid=0525cb73d9a76a060b80df20d87e6b4b)

#### Capturing a few document pages for inspection

Wiley's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [3]:
url_sample_1 = 'https://onlinelibrary.wiley.com/toc/15206300/2020/32/2'
url_sample_11 = 'https://onlinelibrary.wiley.com/doi/10.1002/ajhb.23340'

In [5]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/american_journal_human_biology_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Closing the WebDriver
driver.quit()

In [6]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/american_journal_human_biology_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [3]:
def scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url}{i}"
        driver.get(url)

        # Explicit wait for main article elements
        wait = WebDriverWait(driver, 10)

        # Ensure stable page load
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)

        # Parse page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # Extract Volume/Issue
        volume_issue = f"{volume}, Issue {i}"

        # Extract articles
        articles = soup.find_all('div', class_='issue-item')
        
        for article in articles:
            # Extract article type (if applicable)
            article_type = "Research article"  # Default assumption if not explicitly listed
        
            # Extract title
            title_tag = article.find('a', class_='issue-item__title').find('h2')
            title = title_tag.get_text(strip=True) if title_tag else ''
        
            # Extract article URL
            article_url_tag = article.find('a', class_='issue-item__title')
            article_url = root_url + article_url_tag['href'] if article_url_tag else ''
        
            # Extract authors
            authors_list = article.find('div', class_='comma__list')
            authors = ', '.join([author.get_text(strip=True) for author in authors_list.find_all('span', class_='author-style')]) if authors_list else ''
        
            # Extract published date (removing "First Published: ")
            published_tag = article.find('li', class_='ePubDate')
            published = published_tag.find_all('span')[1].get_text(strip=True) if published_tag else ''
        
            # Extract DOI
            doi_tag = article.find('input', {'type': 'hidden'})
            doi = doi_root_url + doi_tag['value'].replace('%2F', '/') if doi_tag else ''
        
            # Extract Open Access status
            open_access_tag = article.find('div', class_='doi-access')
            open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''
        
            # Extract PDF URL
            pdf_url_tag = article.find('li', class_='PdfLink').find('a')
            pdf_url = pdf_url_tag['href'].replace('epdf', 'pdf') if pdf_url_tag and pdf_url_tag.get('href') else ''
        
            # Append extracted data
            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': published,
                'DOI': doi,
                'Open Access': open_access,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 32 (Issue 1; Issue 2; Issue S1; Issue 3; Issue 4; Issue 5; Issue 6)

Wiley's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [4]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2020/32/'
volume = 'Volume 32'
area_of_knowledge = 'Biological Sciences'

In [5]:
start_page = 1
end_page = 1

In [6]:
df_v32i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.58s/it]


In [7]:
start_page = 2
end_page = 2

In [8]:
df_v32i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.27s/it]


In [9]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2020/32/S'
volume = 'Volume 32'
area_of_knowledge = 'Biological Sciences'

In [10]:
start_page = 1
end_page = 1

In [11]:
df_v32is1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:11<00:00, 11.53s/it]


In [12]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2020/32/'
volume = 'Volume 32'
area_of_knowledge = 'Biological Sciences'

In [13]:
start_page = 3
end_page = 3

In [14]:
df_v32i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:17<00:00, 17.23s/it]


In [15]:
start_page = 4
end_page = 4

In [16]:
df_v32i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.40s/it]


In [17]:
start_page = 5
end_page = 5

In [18]:
df_v32i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.34s/it]


In [19]:
start_page = 6
end_page = 6

In [20]:
df_v32i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.72s/it]


##### Volume 33 (Issue 1; Issue S1; Issue 2; Issue 3; Issue 4; Issue 5; Issue 6)

In [21]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2021/33/'
volume = 'Volume 33'
area_of_knowledge = 'Biological Sciences'

In [22]:
start_page = 1
end_page = 1

In [23]:
df_v33i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.27s/it]


In [24]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2021/33/S'
volume = 'Volume 33'
area_of_knowledge = 'Biological Sciences'

In [25]:
start_page = 1
end_page = 1

In [26]:
df_v33is1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.68s/it]


In [27]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2021/33/'
volume = 'Volume 33'
area_of_knowledge = 'Biological Sciences'

In [28]:
start_page = 2
end_page = 2

In [29]:
df_v33i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.28s/it]


In [30]:
start_page = 3
end_page = 3

In [31]:
df_v33i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:14<00:00, 14.61s/it]


In [32]:
start_page = 4
end_page = 4

In [33]:
df_v33i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.18s/it]


In [34]:
start_page = 5
end_page = 5

In [35]:
df_v33i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.59s/it]


In [36]:
start_page = 6
end_page = 6

In [37]:
df_v33i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.78s/it]


##### Volume 34 (Issue 1; Issue S1; Issue 2; Issue S2; Issue 3; Issue 4; Issue 5; Issue 6; Issue 7; Issue 8; Issue 9; Issue 10; Issue 11; Issue 12)

In [38]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2022/34/'
volume = 'Volume 34'
area_of_knowledge = 'Biological Sciences'

In [39]:
start_page = 1
end_page = 1

In [40]:
df_v34i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:14<00:00, 14.29s/it]


In [41]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2022/34/S'
volume = 'Volume 34'
area_of_knowledge = 'Biological Sciences'

In [42]:
start_page = 1
end_page = 1

In [43]:
df_v34is1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.42s/it]


In [44]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2022/34/'
volume = 'Volume 34'
area_of_knowledge = 'Biological Sciences'

In [45]:
start_page = 2
end_page = 2

In [46]:
df_v34i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:16<00:00, 16.83s/it]


In [47]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2022/34/S'
volume = 'Volume 34'
area_of_knowledge = 'Biological Sciences'

In [48]:
start_page = 2
end_page = 2

In [49]:
df_v34is2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:14<00:00, 14.44s/it]


In [50]:
root_url = 'https://onlinelibrary.wiley.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://onlinelibrary.wiley.com/toc/15206300/2022/34/'
volume = 'Volume 34'
area_of_knowledge = 'Biological Sciences'

In [51]:
start_page = 3
end_page = 3

In [52]:
df_v34i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:18<00:00, 18.07s/it]


In [53]:
start_page = 4
end_page = 4

In [54]:
df_v34i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:14<00:00, 14.16s/it]


In [55]:
start_page = 5
end_page = 5

In [56]:
df_v34i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.36s/it]


In [57]:
start_page = 6
end_page = 6

In [58]:
df_v34i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.32s/it]


In [59]:
start_page = 7
end_page = 7

In [60]:
df_v34i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.21s/it]


In [61]:
start_page = 8
end_page = 8

In [62]:
df_v34i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.86s/it]


In [63]:
start_page = 9
end_page = 9

In [64]:
df_v34i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.20s/it]


In [65]:
start_page = 10
end_page = 10

In [66]:
df_v34i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 13.00s/it]


In [67]:
start_page = 11
end_page = 11

In [68]:
df_v34i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:15<00:00, 15.29s/it]


In [69]:
start_page = 12
end_page = 12

In [70]:
df_v34i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:13<00:00, 13.05s/it]


##### Concatenating the DataFrames

In [71]:
df_american_journal_human_biology = pd.concat([
    df_v32i1, df_v32i2, df_v32is1, df_v32i3, df_v32i4, df_v32i5, df_v32i6,
    df_v33i1, df_v33is1, df_v33i2, df_v33i3, df_v33i4, df_v33i5, df_v33i6,
    df_v34i1, df_v34is1, df_v34i2, df_v34is2, df_v34i3, df_v34i4, df_v34i5, df_v34i6, df_v34i7, df_v34i8, df_v34i9, df_v34i10, df_v34i11, df_v34i12
], ignore_index=True)

In [72]:
df_american_journal_human_biology['Published'] = pd.to_datetime(
    df_american_journal_human_biology['Published'], format='mixed', errors='coerce'
)

In [73]:
df_american_journal_human_biology.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [74]:
unique_open_access = df_american_journal_human_biology['Open Access'].dropna().unique()
unique_open_access

array(['Free Access', '', 'Open Access', 'Free to Read'], dtype=object)

In [75]:
df_american_journal_human_biology_open_access = df_american_journal_human_biology[
    (df_american_journal_human_biology['Open Access'] == 'Open Access') | 
    (df_american_journal_human_biology['Open Access'] == 'Free Access')
]
df_american_journal_human_biology_open_access = df_american_journal_human_biology_open_access[df_american_journal_human_biology_open_access['Published'] <= '2022-11-30']
df_american_journal_human_biology_open_access = df_american_journal_human_biology_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [76]:
df_american_journal_human_biology_open_access = df_american_journal_human_biology_open_access.loc[
    ~df_american_journal_human_biology_open_access['Title'].isin(['Issue Information', 'Erratum'])
].reset_index(drop=True)

In [77]:
df_american_journal_human_biology_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,PDF URL,Area of Knowledge
0,Research article,Water & Human Biologyand New Developments for ...,https://onlinelibrary.wiley.com/doi/10.1002/aj...,William R. Leonard,"Volume 32, Issue 1",2020-01-09,https://doi.org/10.1002/ajhb.23389,Free Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
1,Research article,Measuring Human Water Needs,https://onlinelibrary.wiley.com/doi/10.1002/aj...,"Amber Wutich, Asher Y. Rosinger, Justin Stoler...","Volume 32, Issue 1",2019-11-08,https://doi.org/10.1002/ajhb.23350,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
2,Research article,Developmental origins of variability in pelvic...,https://onlinelibrary.wiley.com/doi/10.1002/aj...,"Meghan K. Shirley, Tim J. Cole, Owen J. Arthur...","Volume 32, Issue 2",2019-11-22,https://doi.org/10.1002/ajhb.23340,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
3,Research article,The Xavante Longitudinal Health Study in Brazi...,https://onlinelibrary.wiley.com/doi/10.1002/aj...,"James R. Welch, Aline A. Ferreira, Felipe G. T...","Volume 32, Issue 2",2019-10-26,https://doi.org/10.1002/ajhb.23339,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
4,Research article,Annual Meeting - Human Biology Association,https://onlinelibrary.wiley.com/doi/10.1002/aj...,,"Volume 32, Issue 1",2020-04-01,https://doi.org/10.1002/ajhb.23407,Free Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
...,...,...,...,...,...,...,...,...,...,...
66,Research article,Minimally invasive biomarkers in human and non...,https://onlinelibrary.wiley.com/doi/10.1002/aj...,"Samuel S. Urlacher, Elizabeth Y. Kim, Tiffany ...","Volume 34, Issue 11",2022-10-07,https://doi.org/10.1002/ajhb.23811,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
67,Research article,The utility of dried blood spot measurement of...,https://onlinelibrary.wiley.com/doi/10.1002/aj...,"Maureen J. Devlin, Geeta N. Eick, J. Josh Snod...","Volume 34, Issue 11",2022-10-10,https://doi.org/10.1002/ajhb.23816,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
68,Research article,"Water insecurity, self-reported physical healt...",https://onlinelibrary.wiley.com/doi/10.1002/aj...,"Paula Skye Tallman, Shalean M. Collins, M. Pia...","Volume 34, Issue 12",2022-09-27,https://doi.org/10.1002/ajhb.23805,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences
69,Research article,Is there a difference in student physical acti...,https://onlinelibrary.wiley.com/doi/10.1002/aj...,"Michaela Emily Howells, Katherine L'Amour Wool...","Volume 34, Issue 12",2022-09-16,https://doi.org/10.1002/ajhb.23799,Open Access,https://onlinelibrary.wiley.com/doi/pdf/10.100...,Biological Sciences


#### Inspecting a few articles

**Attention**: Some of the documents might in fact be `Editorials` rather than `Research articles`. Extra refinement is recommended when using the dataset.

In [78]:
df_american_journal_human_biology_open_access.at[0, 'URL']

'https://onlinelibrary.wiley.com/doi/10.1002/ajhb.23389'

#### Exporting to a file

In [79]:
df_american_journal_human_biology_open_access.to_json(f"{output_directory}/american_journal_human_biology_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Reviews Molecular Cell Biology](https://www.nature.com/nrm/)

This journal had no 'Open Access' articles at the time of data collection.

## Engineering

This discipline has been excluded from the study.

### Scraping [Advanced Materials](https://onlinelibrary.wiley.com/journal/15214095?msockid=0525cb73d9a76a060b80df20d87e6b4b)

### Scraping [IEEE Communications Surveys and Tutorials](https://ieeexplore.ieee.org/document/9621329)

### Scraping [IEEE Transactions on Robotics](https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=8860)

### Scraping [Nature Materials](https://www.nature.com/nmat/)

#### Scraping the articles

In [17]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/nmat/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 156
area_of_knowledge = 'Engineering'

In [18]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_materials = pd.DataFrame(data)

100%|██████████| 156/156 [04:08<00:00,  1.60s/it]


In [19]:
df_nature_materials['Published'] = pd.to_datetime(df_nature_materials['Published'])
df_nature_materials

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Spin switching in Sm0.7Er0.3FeO3triggered by t...,https://www.nature.com/articles/s41563-024-020...,"Zhenya Zhang, Minoru Kanega, Hideki Hirori",2024-10-25,https://www.nature.com/articles/s41563-024-020...,,Engineering
1,"Formation, chemical evolution and solidificati...",https://www.nature.com/articles/s41563-024-020...,"Biao Jin, Ying Chen, James J. De Yoreo",2024-10-24,https://www.nature.com/articles/s41563-024-020...,,Engineering
2,Antagonistic-contracting high-power photo-osci...,https://www.nature.com/articles/s41563-024-020...,"Yusen Zhao, Zixiao Liu, Ximin He",2024-10-24,https://www.nature.com/articles/s41563-024-020...,,Engineering
3,Obtaining V2(PO4)3by sodium extraction from si...,https://www.nature.com/articles/s41563-024-020...,"Sunkyu Park, Ziliang Wang, Christian Masquelier",2024-10-23,https://www.nature.com/articles/s41563-024-020...,,Engineering
4,Coherent optical spin Hall transport for polar...,https://www.nature.com/articles/s41563-024-020...,"Ying Shi, Yusong Gan, Qihua Xiong",2024-10-22,https://www.nature.com/articles/s41563-024-020...,,Engineering
...,...,...,...,...,...,...,...
3101,Bridge-mediated hopping or superexchange elect...,https://www.nature.com/articles/nmat706,"Christoph Lambert, Gilbert Nöll, Jürgen Schelter",2002-09-02,https://www.nature.com/articles/nmat706.pdf,,Engineering
3102,Polymer-stabilized liquid crystal blue phases,https://www.nature.com/articles/nmat712,"Hirotsugu Kikuchi, Masayuki Yokota, Tisato Kaj...",2002-09-02,https://www.nature.com/articles/nmat712.pdf,,Engineering
3103,Closed-loop phase behaviour in block copolymers,https://www.nature.com/articles/nmat724,"Du Yeol Ryu, Unyong Jeong, Thomas P. Russell",2002-09-02,https://www.nature.com/articles/nmat724.pdf,,Engineering
3104,In situepitaxial MgB2thin films for supercondu...,https://www.nature.com/articles/nmat703,"Xianghui Zeng, Alexej V. Pogrebnyakov, Zi-Kui Liu",2002-09-02,https://www.nature.com/articles/nmat703.pdf,,Engineering


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [20]:
df_nature_materials_open_access = df_nature_materials[df_nature_materials['Open Access'] == 'Open Access']
df_nature_materials_open_access = df_nature_materials_open_access[df_nature_materials_open_access['Published'] <= '2022-11-30']
df_nature_materials_open_access = df_nature_materials_open_access.reset_index(drop=True)
df_nature_materials_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Exchange controlled triplet fusion in metal–or...,https://www.nature.com/articles/s41563-022-013...,"Dong-Gwang Ha, Ruomeng Wan, Mircea Dincă",2022-10-06,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
1,Robust cholesteric liquid crystal elastomer fi...,https://www.nature.com/articles/s41563-022-013...,"Yong Geng, Rijeesh Kizhakidathazhath, Jan P. F...",2022-09-29,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
2,Giant voltage amplification from electrostatic...,https://www.nature.com/articles/s41563-022-013...,"Mónica Graf, Hugo Aramberri, Jorge Íñiguez",2022-08-25,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
3,Cell clusters softening triggers collective ce...,https://www.nature.com/articles/s41563-022-013...,"Cristian L. Marchant, Abdul N. Malmi-Kakkada, ...",2022-08-15,https://www.nature.com/articles/s41563-022-013...,Open Access,Engineering
4,Unconventional excitonic states with phonon si...,https://www.nature.com/articles/s41563-022-012...,"Ling Zhou, Junwei Huang, Hongtao Yuan",2022-06-16,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
5,Programmable gear-based mechanical metamaterials,https://www.nature.com/articles/s41563-022-012...,"Xin Fang, Jihong Wen, Peter Gumbsch",2022-06-09,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
6,3D printed protein-based robotic structures ac...,https://www.nature.com/articles/s41563-022-012...,"Haiyang Jia, Johannes Flommersfeld, Petra Schw...",2022-05-26,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
7,STING agonist delivery by tumour-penetrating P...,https://www.nature.com/articles/s41563-022-012...,"Eric L. Dane, Alexis Belessiotis-Richards, Dar...",2022-05-23,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
8,Trace removal of benzene vapour using double-w...,https://www.nature.com/articles/s41563-022-012...,"Tao He, Xiang-Jing Kong, Jian-Rong Li",2022-04-28,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering
9,"Superconductivity in (Ba,K)SbO3",https://www.nature.com/articles/s41563-022-012...,"Minu Kim, Graham M. McNally, Hidenori Takagi",2022-02-28,https://www.nature.com/articles/s41563-022-012...,Open Access,Engineering


#### Inspecting a few articles

In [21]:
df_nature_materials_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41563-022-01368-1.pdf'

#### Exporting to a file

In [22]:
df_nature_materials_open_access.to_json(f"{output_directory}/nature_materials_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Nanotechnology](https://www.nature.com/nnano/)

#### Scraping the articles

In [23]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/nnano/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 115
area_of_knowledge = 'Engineering'

In [24]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_nanotechnology = pd.DataFrame(data)

100%|██████████| 115/115 [03:01<00:00,  1.57s/it]


In [25]:
df_nature_nanotechnology['Published'] = pd.to_datetime(df_nature_nanotechnology['Published'])
df_nature_nanotechnology

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Synthetic syntrophy for adenine nucleotide cro...,https://www.nature.com/articles/s41565-024-018...,"Laura Heinen, Marco van den Noort, Bert Poolman",2024-10-21,https://www.nature.com/articles/s41565-024-018...,Open Access,Engineering
1,Out-of-plane coordination of iridium single at...,https://www.nature.com/articles/s41565-024-018...,"Jie Zhao, Yue Guo, Francesco Ciucci",2024-10-21,https://www.nature.com/articles/s41565-024-018...,Open Access,Engineering
2,Linearly programmable two-dimensional halide p...,https://www.nature.com/articles/s41565-024-017...,"Seung Ju Kim, In Hyuk Im, Ho Won Jang",2024-10-18,https://www.nature.com/articles/s41565-024-017...,,Engineering
3,Magnetoelectric nanodiscs enable wireless tran...,https://www.nature.com/articles/s41565-024-017...,"Ye Ji Kim, Noah Kent, Polina Anikeeva",2024-10-11,https://www.nature.com/articles/s41565-024-017...,Open Access,Engineering
4,Rapid precision targeting of nanoparticles to ...,https://www.nature.com/articles/s41565-024-017...,"Tapas R. Nayak, Adrian Chrastina, Jan E. Schni...",2024-10-08,https://www.nature.com/articles/s41565-024-017...,,Engineering
...,...,...,...,...,...,...,...
2280,A nanoplasmonic molecular ruler for measuring ...,https://www.nature.com/articles/nnano.2006.51,"Gang L. Liu, Yadong Yin, Fanqing Frank Chen",2006-10-04,https://www.nature.com/articles/nnano.2006.51.pdf,,Engineering
2281,Giant piezoresistance effect in silicon nanowires,https://www.nature.com/articles/nnano.2006.53,"Rongrui He, Peidong Yang",2006-10-04,https://www.nature.com/articles/nnano.2006.53.pdf,,Engineering
2282,Digital memory device based on tobacco mosaic ...,https://www.nature.com/articles/nnano.2006.55,"Ricky J. Tseng, Chunglin Tsai, Yang Yang",2006-10-04,https://www.nature.com/articles/nnano.2006.55.pdf,,Engineering
2283,Torsional electromechanical quantum oscillatio...,https://www.nature.com/articles/nnano.2006.57,"Tzahi Cohen-Karni, Lior Segev, Ernesto Joselevich",2006-10-04,https://www.nature.com/articles/nnano.2006.57.pdf,,Engineering


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [26]:
df_nature_nanotechnology_open_access = df_nature_nanotechnology[df_nature_nanotechnology['Open Access'] == 'Open Access']
df_nature_nanotechnology_open_access = df_nature_nanotechnology_open_access[df_nature_nanotechnology_open_access['Published'] <= '2022-11-30']
df_nature_nanotechnology_open_access = df_nature_nanotechnology_open_access.reset_index(drop=True)
df_nature_nanotechnology_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Potential use of engineered nanoparticles in o...,https://www.nature.com/articles/s41565-022-012...,"Peyman Babakhani, Tanapon Phenrat, Michael F. ...",2022-11-28,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
1,Feedback-controlled hydrogels with homeostatic...,https://www.nature.com/articles/s41565-022-012...,"Hang Zhang, Hao Zeng, Olli Ikkala",2022-11-28,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
2,Quantum Hall phase in graphene engineered by i...,https://www.nature.com/articles/s41565-022-012...,"Yaning Wang, Xiang Gao, Zhidong Zhang",2022-11-21,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
3,Hierarchically self-assembled homochiral helic...,https://www.nature.com/articles/s41565-022-012...,"Cong Du, Zujian Li, Minghua Liu",2022-11-03,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
4,Human-muscle-inspired single fibre actuator wi...,https://www.nature.com/articles/s41565-022-012...,"In Ho Kim, Subi Choi, Sang Ouk Kim",2022-10-27,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
5,Nano-optical designs for high-efficiency monol...,https://www.nature.com/articles/s41565-022-012...,"Philipp Tockhorn, Johannes Sutter, Christiane ...",2022-10-24,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
6,Local and global energy barriers for chiral do...,https://www.nature.com/articles/s41565-022-012...,"Jiho Yoon, See-Hun Yang, Stuart. S. P. Parkin",2022-10-06,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
7,A single hole spin with enhanced coherence in ...,https://www.nature.com/articles/s41565-022-011...,"N. Piot, B. Brun, S. De Franceschi",2022-09-22,https://www.nature.com/articles/s41565-022-011...,Open Access,Engineering
8,Three-dimensional racetrack memory devices des...,https://www.nature.com/articles/s41565-022-012...,"Ke Gu, Yicheng Guan, Stuart S. P. Parkin",2022-09-22,https://www.nature.com/articles/s41565-022-012...,Open Access,Engineering
9,Doping-driven topological polaritons in graphe...,https://www.nature.com/articles/s41565-022-011...,"Hai Hu, Na Chen, Qing Dai",2022-08-18,https://www.nature.com/articles/s41565-022-011...,Open Access,Engineering


#### Inspecting a few articles

In [27]:
df_nature_nanotechnology_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41565-022-01226-w.pdf'

#### Exporting to a file

In [28]:
df_nature_nanotechnology_open_access.to_json(f"{output_directory}/nature_nanotechnology_open_access.jsonl", orient='records', lines=True)

## Exact and Earth Sciences

This discipline has been excluded from the study.

### Scraping [Annual Review of Earth and Planetary Sciences](https://www.annualreviews.org/content/journals/earth)

#### Scraping the articles

##### Function to scrape the articles

Make sure you run the function `scrape_articles` on section `Scraping Annual Review of Plant Biology` before scraping the articles.

##### Volume 50, 2022

In [15]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/earth/50/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Exact and Earth Sciences'

In [16]:
df_2022 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:09<00:00,  4.50s/it]


##### Volume 49, 2021

In [17]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/earth/49/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Exact and Earth Sciences'

In [18]:
df_2021 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:10<00:00,  5.07s/it]


##### Volume 48, 2020

In [19]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/earth/48/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Exact and Earth Sciences'

In [20]:
df_2020 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:07<00:00,  3.83s/it]


##### Concatenating the DataFrames

In [21]:
df_ar_earth_planetary = pd.concat([df_2022, df_2021, df_2020], ignore_index=True)
df_ar_earth_planetary

,Title,URL,Authors,Vol/Year/Page Range,DOI,Area of Knowledge
0,Introduction,https://www.annualreviews.org/content/journals...,"Katherine H. Freeman, , andRaymond Jeanloz","Vol. 50 \n(2022),pp.v–vii",https://doi.org/10.1146/annurev-ea-50-041822-1...,Exact and Earth Sciences
1,Civilization-Saving Science for the Twenty-Fir...,https://www.annualreviews.org/content/journals...,Marcia K. McNutt,"Vol. 50 \n(2022),pp.1–12",https://doi.org/10.1146/annurev-earth-033021-0...,Exact and Earth Sciences
2,Application of Light Hydrocarbons in Natural G...,https://www.annualreviews.org/content/journals...,"Shipeng Huang, ,Jianzhong Li, ,Tongshan Wang, ...","Vol. 50 \n(2022),pp.13–53",https://doi.org/10.1146/annurev-earth-070921-0...,Exact and Earth Sciences
3,Where Has All the Carbon Gone?,https://www.annualreviews.org/content/journals...,A. Scott Denning,"Vol. 50 \n(2022),pp.55–78",https://doi.org/10.1146/annurev-earth-032320-0...,Exact and Earth Sciences
4,Volcanic Outgassing of Volatile Trace Metals,https://www.annualreviews.org/content/journals...,"Marie Edmonds, ,Emily Mason, , andOlivia Hogg","Vol. 50 \n(2022),pp.79–98",https://doi.org/10.1146/annurev-earth-070921-0...,Exact and Earth Sciences
...,...,...,...,...,...,...
70,Climate Extremes and Compound Hazards in a War...,https://www.annualreviews.org/content/journals...,"Amir AghaKouchak, ,Felicia Chiang, ,Laurie S. ...","Vol. 48 \n(2020),pp.519–548",https://doi.org/10.1146/annurev-earth-071719-0...,Exact and Earth Sciences
71,The Role of Diagenesis in Shaping the Geochemi...,https://www.annualreviews.org/content/journals...,"Matthew S. Fantle, ,B. Davis Barnes, , andKimb...","Vol. 48 \n(2020),pp.549–583",https://doi.org/10.1146/annurev-earth-073019-0...,Exact and Earth Sciences
72,Dynamic Topography and Ice Age Paleoclimate,https://www.annualreviews.org/content/journals...,"J.X. Mitrovica, ,J. Austermann, ,S. Coulson, ,...","Vol. 48 \n(2020),pp.585–621",https://doi.org/10.1146/annurev-earth-082517-0...,Exact and Earth Sciences
73,Moist Heat Stress on a Hotter Earth,https://www.annualreviews.org/content/journals...,"Jonathan R. Buzan, , andMatthew Huber","Vol. 48 \n(2020),pp.623–655",https://doi.org/10.1146/annurev-earth-053018-0...,Exact and Earth Sciences


#### Inspecting a few articles

**Attention**: Some of the documents might in fact be `Editorials` rather than `Research articles`. Extra refinement is recommended when using the dataset.

In [22]:
df_ar_earth_planetary.at[0, 'URL']

'https://www.annualreviews.org/content/journals/10.1146/annurev-ea-50-041822-100001'

#### Exporting to a file

In [23]:
df_ar_earth_planetary.to_json(f"{output_directory}/ar_earth_planetary.jsonl", orient='records', lines=True)

### Scraping [Journal of Geophysical Research: Solid Earth](https://agupubs.onlinelibrary.wiley.com/journal/21699356)

### Scraping [Nature Geoscience](https://www.nature.com/ngeo/)

#### Scraping the articles

In [29]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/ngeo/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 115
area_of_knowledge = 'Exact and Earth Sciences'

In [30]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_geoscience = pd.DataFrame(data)

100%|██████████| 115/115 [03:04<00:00,  1.61s/it]


In [31]:
df_nature_geoscience['Published'] = pd.to_datetime(df_nature_geoscience['Published'])
df_nature_geoscience

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Enhanced ocean CO2uptake due to near-surface t...,https://www.nature.com/articles/s41561-024-015...,"Daniel J. Ford, Jamie D. Shutler, Ian Ashton",2024-10-25,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
1,Direct and lagged climate change effects inten...,https://www.nature.com/articles/s41561-024-015...,"Emanuele Bevacqua, Oldrich Rakovec, Jakob Zsch...",2024-10-21,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
2,Large-scale mantle heterogeneity as a legacy o...,https://www.nature.com/articles/s41561-024-015...,"Luc S. Doucet, Zheng-Xiang Li",2024-10-18,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
3,Centennial-scale variations in the carbon cycl...,https://www.nature.com/articles/s41561-024-015...,"Etienne Legrain, Emilie Capron, Thomas F. Stocker",2024-10-11,https://www.nature.com/articles/s41561-024-015...,,Exact and Earth Sciences
4,Nitrogen fixation in the North Atlantic suppor...,https://www.nature.com/articles/s41561-024-015...,"Cora Hoerstmann, Borja Aguiar-González, Mar Be...",2024-10-11,https://www.nature.com/articles/s41561-024-015...,Open Access,Exact and Earth Sciences
...,...,...,...,...,...,...,...
2284,High rates of sea-level rise during the last i...,https://www.nature.com/articles/ngeo.2007.28,"E. J. Rohling, K. Grant, M. Kucera",2007-12-16,https://www.nature.com/articles/ngeo.2007.28.pdf,,Exact and Earth Sciences
2285,Persistent earthquake clusters and gaps from s...,https://www.nature.com/articles/ngeo.2007.36,Tom Parsons,2007-12-09,https://www.nature.com/articles/ngeo.2007.36.pdf,,Exact and Earth Sciences
2286,Atmospheric carbon dioxide linked with Mesozoi...,https://www.nature.com/articles/ngeo.2007.29,"Benjamin J. Fletcher, Stuart J. Brentnall, Dav...",2007-12-09,https://www.nature.com/articles/ngeo.2007.29.pdf,,Exact and Earth Sciences
2287,Influence of brine formation on Arctic Ocean c...,https://www.nature.com/articles/ngeo.2007.5,"Brian A. Haley, Martin Frank, Anton Eisenhauer",2007-12-02,https://www.nature.com/articles/ngeo.2007.5.pdf,,Exact and Earth Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [32]:
df_nature_geoscience_open_access = df_nature_geoscience[df_nature_geoscience['Open Access'] == 'Open Access']
df_nature_geoscience_open_access = df_nature_geoscience_open_access[df_nature_geoscience_open_access['Published'] <= '2022-11-30']
df_nature_geoscience_open_access = df_nature_geoscience_open_access.reset_index(drop=True)
df_nature_geoscience_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Limited carbon cycling due to high-pressure ef...,https://www.nature.com/articles/s41561-022-010...,"Chie Amano, Zihao Zhao, Gerhard J. Herndl",2022-11-28,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
1,Global patterns in marine organic matter stoic...,https://www.nature.com/articles/s41561-022-010...,"Keisuke Inomura, Curtis Deutsch, Michael J. Fo...",2022-11-21,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
2,Carbon dioxide sink in the Arctic Ocean from c...,https://www.nature.com/articles/s41561-022-010...,"Andreas Rogge, Markus Janout, Anya M. Waite",2022-11-21,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
3,Fraser Island (K'gari) and initiation of the G...,https://www.nature.com/articles/s41561-022-010...,"D. Ellerton, T. M. Rittenour, X. Zhao",2022-11-14,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
4,Declining metal availability in the Mesozoic s...,https://www.nature.com/articles/s41561-022-010...,"Qiong Zhang, El Mahdi Bendif, Rosalind E. M. R...",2022-10-31,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
5,Millennial-scale climate variability over land...,https://www.nature.com/articles/s41561-022-010...,"R. Hébert, U. Herzschuh, T. Laepple",2022-10-31,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
6,Long-range predictability of extratropical cli...,https://www.nature.com/articles/s41561-022-010...,"A. A. Scaife, L. Hermanson, D. Smith",2022-10-03,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
7,Submarine melting of glaciers in Greenland amp...,https://www.nature.com/articles/s41561-022-010...,"D. A. Slater, F. Straneo",2022-10-03,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
8,Carbon and sediment fluxes inhibited in the su...,https://www.nature.com/articles/s41561-022-010...,"Ed L. Pope, Maarten S. Heijnen, Morelia Urlaub",2022-09-29,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences
9,Pleistocene climate variability in eastern Afr...,https://www.nature.com/articles/s41561-022-010...,"Verena Foerster, Asfawossen Asrat, Martin H. T...",2022-09-26,https://www.nature.com/articles/s41561-022-010...,Open Access,Exact and Earth Sciences


#### Inspecting a few articles

In [33]:
df_nature_geoscience_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41561-022-01081-3.pdf'

#### Exporting to a file

In [34]:
df_nature_geoscience_open_access.to_json(f"{output_directory}/nature_geoscience_open_access.jsonl", orient='records', lines=True)

### Scraping [Reviews of Geophysics](https://agupubs.onlinelibrary.wiley.com/journal/19449208)

## Health Sciences

### Scraping [Nature Medicine](https://www.nature.com/nm/)

#### Scraping the articles

In [10]:
root_url = 'https://www.nature.com'
article_list_url = 'https://www.nature.com/nm/research-articles?searchType=journalSearch&sort=PubDate&page='
start_page = 1
end_page = 241
area_of_knowledge = 'Biological Sciences'

In [11]:
# Initialize an empty list to store the data
data = []

# Iterating through the URLs and using 'tqdm' for progress tracking in the range loop
for i in tqdm(range(start_page, end_page + 1)):
    url = f"{article_list_url}{i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml')

    # Finding all <article> elements with the class 'u-full-height c-card c-card--flush'
    articles = soup.find_all('article', class_='u-full-height c-card c-card--flush')

    for article in articles:
        # Extracting the title
        title_tag = article.find('h3', class_='c-card__title').find('a')
        title = title_tag.get_text(strip=True) if title_tag else ''
        title_url = f"{root_url}{title_tag.get('href')}" if title_tag else ''

        # Extracting the authors
        author_tags = article.find_all('li', itemprop='creator')
        authors = ', '.join(author.get_text(strip=True) for author in author_tags)

        # Extracting the published date
        date_tag = article.find('time')
        date_published = date_tag['datetime'] if date_tag else ''

        # Extracting the PDF URL
        pdf_url = f"{title_url}.pdf" if title_url else ''

        # Extracting the 'Open Access' label
        open_access_tag = article.find('span', class_='u-color-open-access')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

        # Appending the data to the list
        data.append({
            'Title': title,
            'URL': title_url,
            'Authors (compact list)': authors,
            'Published': date_published,
            'PDF URL': pdf_url,
            'Open Access': open_access,
            'Area of Knowledge': area_of_knowledge
        })

# Creating a DataFrame from the data
df_nature_medicine = pd.DataFrame(data)

100%|██████████| 241/241 [06:31<00:00,  1.62s/it]


In [12]:
df_nature_medicine['Published'] = pd.to_datetime(df_nature_medicine['Published'])
df_nature_medicine

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,Semaglutide in patients with overweight or obe...,https://www.nature.com/articles/s41591-024-033...,"Ellen M. Apperloo, Jose L. Gorriz, Hiddo J. L....",2024-10-25,https://www.nature.com/articles/s41591-024-033...,,Biological Sciences
1,Subclassification of obesity for precision pre...,https://www.nature.com/articles/s41591-024-032...,"Daniel E. Coral, Femke Smit, Paul W. Franks",2024-10-24,https://www.nature.com/articles/s41591-024-032...,Open Access,Biological Sciences
2,Global genomic surveillance of monkeypox virus,https://www.nature.com/articles/s41591-024-033...,"James R. Otieno, Christopher Ruis, Lorenzo Sub...",2024-10-23,https://www.nature.com/articles/s41591-024-033...,,Biological Sciences
3,Home-based transcranial direct current stimula...,https://www.nature.com/articles/s41591-024-033...,"Rachel D. Woodham, Sudhakar Selvaraj, Cynthia ...",2024-10-21,https://www.nature.com/articles/s41591-024-033...,Open Access,Biological Sciences
4,AAV gene therapy for Duchenne muscular dystrop...,https://www.nature.com/articles/s41591-024-033...,"Jerry R. Mendell, Francesco Muntoni, Louise R....",2024-10-09,https://www.nature.com/articles/s41591-024-033...,Open Access,Biological Sciences
...,...,...,...,...,...,...,...
4804,Microsatellite instability in primary neoplasm...,https://www.nature.com/articles/nm0195-65,"Gauri C. Bedi, William H. Westra, David Sidransky",1995-01-01,https://www.nature.com/articles/nm0195-65.pdf,,Biological Sciences
4805,Serial magnetic resonance imaging of experimen...,https://www.nature.com/articles/nm0195-69,"Michael P. Skinner, Chun Yuan, Russell Ross",1995-01-01,https://www.nature.com/articles/nm0195-69.pdf,,Biological Sciences
4806,Liposome-mediatedCFTRgene transfer to the nasa...,https://www.nature.com/articles/nm0195-39,"Natasha J. Caplen, Eric W.F.W. Alton, Duncan M...",1995-01-01,https://www.nature.com/articles/nm0195-39.pdf,,Biological Sciences
4807,"The paternal inheritance of the centrosome, th...",https://www.nature.com/articles/nm0195-47,"Calvin Simerly, Gwo-Jang Wu, Gerald Schatten",1995-01-01,https://www.nature.com/articles/nm0195-47.pdf,,Biological Sciences


#### Selecting the 'Open Access' articles published before the advent of ChatGPT

ChatGPT was released by OpenAI on November 30, 2022.

In [13]:
df_nature_medicine_open_access = df_nature_medicine[df_nature_medicine['Open Access'] == 'Open Access']
df_nature_medicine_open_access = df_nature_medicine_open_access[df_nature_medicine_open_access['Published'] <= '2022-11-30']
df_nature_medicine_open_access = df_nature_medicine_open_access.reset_index(drop=True)
df_nature_medicine_open_access

,Title,URL,Authors (compact list),Published,PDF URL,Open Access,Area of Knowledge
0,CSF tau microtubule-binding region identifies ...,https://www.nature.com/articles/s41591-022-020...,"Kanta Horie, Nicolas R. Barthélemy, Chihiro Sato",2022-11-24,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
1,Global and regional estimates of orphans attri...,https://www.nature.com/articles/s41591-022-021...,"Florence Guida, Rachel Kidman, Valerie McCormack",2022-11-20,https://www.nature.com/articles/s41591-022-021...,Open Access,Biological Sciences
2,Amyloid and tau PET-positive cognitively unimp...,https://www.nature.com/articles/s41591-022-020...,"Rik Ossenkoppele, Alexa Pichet Binette, Oskar ...",2022-11-10,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
3,Acute and postacute sequelae associated with S...,https://www.nature.com/articles/s41591-022-020...,"Benjamin Bowe, Yan Xie, Ziyad Al-Aly",2022-11-10,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
4,Rare and common genetic determinants of metabo...,https://www.nature.com/articles/s41591-022-020...,"Praveen Surendran, Isobel D. Stewart, Claudia ...",2022-11-10,https://www.nature.com/articles/s41591-022-020...,Open Access,Biological Sciences
...,...,...,...,...,...,...,...
171,Investigation of the role of typhoid toxin in ...,https://www.nature.com/articles/s41591-019-0505-4,"Malick M. Gibani, Elizabeth Jones, Andrew J. P...",2019-07-03,https://www.nature.com/articles/s41591-019-050...,Open Access,Biological Sciences
172,The vaginal microbiome and preterm birth,https://www.nature.com/articles/s41591-019-0450-2,"Jennifer M. Fettweis, Myrna G. Serrano, Gregor...",2019-05-29,https://www.nature.com/articles/s41591-019-045...,Open Access,Biological Sciences
173,IFN-γ-independent immune markers ofMycobacteri...,https://www.nature.com/articles/s41591-019-0441-3,"Lenette L. Lu, Malisa T. Smith, Galit Alter",2019-05-20,https://www.nature.com/articles/s41591-019-044...,Open Access,Biological Sciences
174,A patient-level pooled analysis of treatment-s...,https://www.nature.com/articles/s41591-018-0224-2,"Marjorie Z. Imperial, Payam Nahid, Rada M. Savic",2018-11-05,https://www.nature.com/articles/s41591-018-022...,Open Access,Biological Sciences


#### Inspecting a few articles

In [14]:
df_nature_medicine_open_access.at[0, 'PDF URL']

'https://www.nature.com/articles/s41591-022-02075-9.pdf'

#### Exporting to a file

In [16]:
df_nature_medicine_open_access.to_json(f"{output_directory}/nature_medicine_open_access.jsonl", orient='records', lines=True)

### Scraping [Annual Review of Public Health](https://www.annualreviews.org/content/journals/publhealth)

#### Scraping the articles

##### Function to scrape the articles

Make sure you run the function `scrape_articles` on section `Scraping Annual Review of Plant Biology` before scraping the articles.

##### Volume 43, 2022

In [24]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/publhealth/43/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Health Sciences'

In [25]:
df_2022 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:10<00:00,  5.01s/it]


##### Volume 42, 2021

In [26]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/publhealth/42/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Health Sciences'

In [27]:
df_2021 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:08<00:00,  4.20s/it]


##### Volume 41, 2020

In [28]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/publhealth/41/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Health Sciences'

In [29]:
df_2020 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:08<00:00,  4.41s/it]


##### Concatenating the DataFrames

In [30]:
df_ar_public_health = pd.concat([df_2022, df_2021, df_2020], ignore_index=True)
df_ar_public_health

,Title,URL,Authors,Vol/Year/Page Range,DOI,Area of Knowledge
0,Advances in Gender-Transformative Approaches t...,https://www.annualreviews.org/content/journals...,"Jane Fisher, , andShelly Makleff","Vol. 43 \n(2022),pp.1–17",https://doi.org/10.1146/annurev-publhealth-121...,Health Sciences
1,Methods to Address Confounding and Other Biase...,https://www.annualreviews.org/content/journals...,"Maya B. Mathur, , andTyler J. VanderWeele","Vol. 43 \n(2022),pp.19–35",https://doi.org/10.1146/annurev-publhealth-051...,Health Sciences
2,Qualitative Research Methods in Chronic Diseas...,https://www.annualreviews.org/content/journals...,"Rachel C. Shelton, ,Morgan M. Philbin, , andSh...","Vol. 43 \n(2022),pp.37–57",https://doi.org/10.1146/annurev-publhealth-012...,Health Sciences
3,Risks and Opportunities to Ensure Equity in th...,https://www.annualreviews.org/content/journals...,"Paul Wesson, ,Yulin Hswen, ,Gilmer Valdes, ,Kr...","Vol. 43 \n(2022),pp.59–78",https://doi.org/10.1146/annurev-publhealth-051...,Health Sciences
4,"Social Epidemiology: Past, Present, and Future",https://www.annualreviews.org/content/journals...,Ana V. Diez Roux,"Vol. 43 \n(2022),pp.79–98",https://doi.org/10.1146/annurev-publhealth-060...,Health Sciences
...,...,...,...,...,...,...
82,A Public Health Approach to Global Child Sex T...,https://www.annualreviews.org/content/journals...,Jordan Greenbaum,"Vol. 41 \n(2020),pp.481–497",https://doi.org/10.1146/annurev-publhealth-040...,Health Sciences
83,Comparative Approaches to Drug Pricing,https://www.annualreviews.org/content/journals...,"So-Yeon Kang, ,Ge Bai, ,Michael J. DiStefano, ...","Vol. 41 \n(2020),pp.499–512",https://doi.org/10.1146/annurev-publhealth-040...,Health Sciences
84,Disparities in Access to Oral Health Care,https://www.annualreviews.org/content/journals...,"Mary E. Northridge, ,Anjali Kumar, , andRaghbi...","Vol. 41 \n(2020),pp.513–535",https://doi.org/10.1146/annurev-publhealth-040...,Health Sciences
85,"Medicaid Managed Care's Effects on Costs, Acce...",https://www.annualreviews.org/content/journals...,"Daniela Franco Montoya, ,Puneet Kaur Chehal, ,...","Vol. 41 \n(2020),pp.537–549",https://doi.org/10.1146/annurev-publhealth-040...,Health Sciences


#### Inspecting a few articles

**Attention**: Some of the documents might in fact be `Editorials` rather than `Research articles`. Extra refinement is recommended when using the dataset.

In [31]:
df_ar_public_health.at[0, 'URL']

'https://www.annualreviews.org/content/journals/10.1146/annurev-publhealth-121019-053834'

#### Exporting to a file

In [32]:
df_ar_public_health.to_json(f"{output_directory}/ar_public_health.jsonl", orient='records', lines=True)

### Scraping [Lancet Public Health](https://www.thelancet.com/journals/lanpub/home)

#### Capturing a few document pages for inspection

In [3]:
url_sample_1 = 'https://www.thelancet.com/journals/lanpub/issue/vol5no1/PIIS2468-2667(19)X0013-4'
url_sample_11 = 'https://www.thelancet.com/journals/lanpub/article/PIIS2468-2667(19)30230-0/fulltext'

In [4]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/lancet_public_health_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Closing the WebDriver
driver.quit()

In [5]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/lancet_public_health_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [3]:
def scrape_articles(root_url, article_list_url, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    driver.get(article_list_url)

    # Explicit wait for main article elements
    wait = WebDriverWait(driver, 10)

    # Ensure stable page load
    max_wait_time = 30
    start_time = time.time()
    previous_html = ''
    while True:
        current_html = driver.page_source
        if current_html == previous_html or time.time() - start_time > max_wait_time:
            break
        previous_html = current_html
        time.sleep(2)

    # Parse page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # Extract Published/Volume/Issue
    page_title_tag = soup.find('meta', {'name': 'name'})
    page_title = page_title_tag.get('content', '') if page_title_tag else ''
    published = page_title.split(',')[1].strip() if ',' in page_title else ''
    volume = page_title.split('Volume ')[-1].split(',')[0].strip() if 'Volume' in page_title else ''
    issue = page_title.split('Issue ')[-1].split(',')[0].strip() if 'Issue' in page_title else ''

    # Extract articles
    articles = []
    articles_header = soup.find('h2', string='Articles')
    if articles_header:
        articles_list = articles_header.find_next('ul', class_='toc__body')
        if articles_list:
            articles = articles_list.find_all('li', class_='articleCitation')

    for article in articles:
        # Extract article type
        article_type = 'Article'
    
        # Extract title
        title_tag = article.find('h3', class_='toc__item__title')
        title = title_tag.get_text(strip=True) if title_tag else ''
    
        # Extract article URL
        article_url_tag = title_tag.find('a') if title_tag else None
        article_url = f"{root_url}{article_url_tag.get('href', '')}" if article_url_tag else ''
    
        # Extract authors
        authors_list = article.find('ul', class_='toc__item__authors')
        authors = ', '.join([author.get_text(strip=True).rstrip(',') for author in authors_list.find_all('li', class_='loa__item')]) if authors_list else ''
    
        # Extract PDF URL
        pdf_link_tag = article.find('a', class_='pdfLink')
        pdf_url = f"{root_url}{pdf_link_tag.get('href', '')}" if pdf_link_tag else ''
    
        # Append extracted data
        data.append({
            'Article Type': article_type,
            'Title': title,
            'URL': article_url,
            'Authors': authors,
            'Vol/Issue': f"Volume {volume}/Issue {issue}",
            'Published': published,
            'PDF URL': pdf_url,
            'Area of Knowledge': area_of_knowledge
        })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 5 (Issue 1 - 12)

The Lancet Public Health's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [4]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no1/PIIS2468-2667(19)X0013-4'
area_of_knowledge = 'Health Sciences'

In [5]:
df_v5i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [6]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no2/PIIS2468-2667(20)X0002-8'
area_of_knowledge = 'Health Sciences'

In [7]:
df_v5i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [8]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no3/PIIS2468-2667(20)X0003-X'
area_of_knowledge = 'Health Sciences'

In [9]:
df_v5i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [10]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no4/PIIS2468-2667(20)X0004-1'
area_of_knowledge = 'Health Sciences'

In [11]:
df_v5i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [12]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no5/PIIS2468-2667(20)X0005-3'
area_of_knowledge = 'Health Sciences'

In [13]:
df_v5i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [14]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no6/PIIS2468-2667(20)X0006-5'
area_of_knowledge = 'Health Sciences'

In [15]:
df_v5i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [16]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no7/PIIS2468-2667(20)X0007-7'
area_of_knowledge = 'Health Sciences'

In [17]:
df_v5i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [18]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no8/PIIS2468-2667(20)X0008-9'
area_of_knowledge = 'Health Sciences'

In [19]:
df_v5i8 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [20]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no9/PIIS2468-2667(20)X0009-0'
area_of_knowledge = 'Health Sciences'

In [21]:
df_v5i9 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [22]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no10/PIIS2468-2667(20)X0010-7'
area_of_knowledge = 'Health Sciences'

In [23]:
df_v5i10 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [24]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no11/PIIS2468-2667(20)X0011-9'
area_of_knowledge = 'Health Sciences'

In [25]:
df_v5i11 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [26]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol5no12/PIIS2468-2667(20)X0012-0'
area_of_knowledge = 'Health Sciences'

In [27]:
df_v5i12 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 6 (Issue 1 - 12)

In [28]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no1/PIIS2468-2667(20)X0013-2'
area_of_knowledge = 'Health Sciences'

In [29]:
df_v6i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [30]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no2/PIIS2468-2667(21)X0002-3'
area_of_knowledge = 'Health Sciences'

In [31]:
df_v6i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [32]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no3/PIIS2468-2667(21)X0003-5'
area_of_knowledge = 'Health Sciences'

In [33]:
df_v6i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [34]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no4/PIIS2468-2667(21)X0004-7'
area_of_knowledge = 'Health Sciences'

In [35]:
df_v6i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [36]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no5/PIIS2468-2667(21)X0005-9'
area_of_knowledge = 'Health Sciences'

In [37]:
df_v6i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [38]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no6/PIIS2468-2667(21)X0006-0'
area_of_knowledge = 'Health Sciences'

In [39]:
df_v6i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [40]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no7/PIIS2468-2667(21)X0007-2'
area_of_knowledge = 'Health Sciences'

In [41]:
df_v6i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [42]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no8/PIIS2468-2667(21)X0008-4'
area_of_knowledge = 'Health Sciences'

In [43]:
df_v6i8 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [44]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no9/PIIS2468-2667(21)X0009-6'
area_of_knowledge = 'Health Sciences'

In [45]:
df_v6i9 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [46]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no10/PIIS2468-2667(21)X0010-2'
area_of_knowledge = 'Health Sciences'

In [47]:
df_v6i10 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [48]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no11/PIIS2468-2667(21)X0011-4'
area_of_knowledge = 'Health Sciences'

In [49]:
df_v6i11 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [50]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol6no12/PIIS2468-2667(21)X0012-6'
area_of_knowledge = 'Health Sciences'

In [51]:
df_v6i12 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Volume 7 (Issue 1 - 11)

In [52]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no1/PIIS2468-2667(21)X0013-8'
area_of_knowledge = 'Health Sciences'

In [53]:
df_v7i1 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [54]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no2/PIIS2468-2667(22)X0002-9'
area_of_knowledge = 'Health Sciences'

In [55]:
df_v7i2 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [56]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no3/PIIS2468-2667(22)X0003-0'
area_of_knowledge = 'Health Sciences'

In [57]:
df_v7i3 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [58]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no4/PIIS2468-2667(22)X0004-2'
area_of_knowledge = 'Health Sciences'

In [59]:
df_v7i4 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [60]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no5/PIIS2468-2667(22)X0005-4'
area_of_knowledge = 'Health Sciences'

In [61]:
df_v7i5 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [62]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no6/PIIS2468-2667(22)X0006-6'
area_of_knowledge = 'Health Sciences'

In [63]:
df_v7i6 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [64]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no7/PIIS2468-2667(22)X0007-8'
area_of_knowledge = 'Health Sciences'

In [65]:
df_v7i7 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [66]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no8/PIIS2468-2667(22)X0008-X'
area_of_knowledge = 'Health Sciences'

In [67]:
df_v7i8 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [68]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no9/PIIS2468-2667(22)X0009-1'
area_of_knowledge = 'Health Sciences'

In [69]:
df_v7i9 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [70]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no10/PIIS2468-2667(22)X0010-8'
area_of_knowledge = 'Health Sciences'

In [71]:
df_v7i10 = scrape_articles(root_url, article_list_url, area_of_knowledge)

In [72]:
root_url = 'https://www.thelancet.com'
article_list_url = 'https://www.thelancet.com/journals/lanpub/issue/vol7no11/PIIS2468-2667(22)X0011-X'
area_of_knowledge = 'Health Sciences'

In [73]:
df_v7i11 = scrape_articles(root_url, article_list_url, area_of_knowledge)

##### Concatenating the DataFrames

In [74]:
df_lancet_public_health = pd.concat([
    df_v5i1, df_v5i2, df_v5i3, df_v5i4, df_v5i5, df_v5i6, df_v5i7, df_v5i8, df_v5i9, df_v5i10, df_v5i11, df_v5i12,
    df_v6i1, df_v6i2, df_v6i3, df_v6i4, df_v6i5, df_v6i6, df_v6i7, df_v6i8, df_v6i9, df_v6i10, df_v6i11, df_v6i12,
    df_v7i1, df_v7i2, df_v7i3, df_v7i4, df_v7i5, df_v7i6, df_v7i7, df_v7i8, df_v7i9, df_v7i10, df_v7i11
], ignore_index=True)

In [76]:
df_lancet_public_health['Published'] = pd.to_datetime(
    df_lancet_public_health['Published'], format='mixed', errors='coerce'
)

In [77]:
df_lancet_public_health.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [79]:
df_lancet_public_health_open_access = df_lancet_public_health
df_lancet_public_health_open_access = df_lancet_public_health_open_access[df_lancet_public_health_open_access['Published'] <= '2022-11-30']
df_lancet_public_health_open_access = df_lancet_public_health_open_access.reset_index(drop=True)

In [80]:
df_lancet_public_health_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,PDF URL,Area of Knowledge
0,Article,Traumatic brain injury in homeless and margina...,https://www.thelancet.com/journals/lanpub/arti...,"Jacob L Stubbs, Allen E Thornton, Jessica M Se...",Volume 5/Issue 1,2020-01-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
1,Article,Premature mortality attributable to socioecono...,https://www.thelancet.com/journals/lanpub/arti...,"Dan Lewer, Wikum Jayatunga, Robert W Aldridge,...",Volume 5/Issue 1,2020-01-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
2,Article,"Social inequalities in multimorbidity, frailty...",https://www.thelancet.com/journals/lanpub/arti...,"Aline Dugravot, Aurore Fayosse, Julien Dumurgi...",Volume 5/Issue 1,2020-01-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
3,Article,"National, regional, and global burdens of dise...",https://www.thelancet.com/journals/lanpub/arti...,"Kevin Shield, Jakob Manthey, Margaret Rylett, ...",Volume 5/Issue 1,2020-01-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
4,Article,"Social disconnectedness, perceived isolation, ...",https://www.thelancet.com/journals/lanpub/arti...,"Ziggi Ivan Santini, Paul E Jose, Erin York Cor...",Volume 5/Issue 1,2020-01-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
...,...,...,...,...,...,...,...,...
173,Article,Effectiveness and cost-effectiveness of referr...,https://www.thelancet.com/journals/lanpub/arti...,"Amy L Ahern, Penny Breeze, Francesco Fusco, St...",Volume 7/Issue 10,2022-10-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
174,Article,The burden of bacterial antimicrobial resistan...,https://www.thelancet.com/journals/lanpub/arti...,European Antimicrobial Resistance Collaborators,Volume 7/Issue 11,2022-11-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
175,Article,Association between influenza vaccination and ...,https://www.thelancet.com/journals/lanpub/arti...,"Jessalyn K Holodinsky, Charlotte Zerna, Shaun ...",Volume 7/Issue 11,2022-11-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences
176,Article,Educational inequalities and premature mortali...,https://www.thelancet.com/journals/lanpub/arti...,"Stephanie Ross, Nurys Armas Rojas, Julia Sawat...",Volume 7/Issue 11,2022-11-01,https://www.thelancet.com/pdfs/journals/lanpub...,Health Sciences


#### Inspecting a few articles

In [81]:
df_lancet_public_health_open_access.at[4, 'URL']

'https://www.thelancet.com/journals/lanpub/article/PIIS2468-2667(19)30230-0/fulltext'

#### Exporting to a file

In [82]:
df_lancet_public_health_open_access.to_json(f"{output_directory}/lancet_public_health_open_access.jsonl", orient='records', lines=True)

### Scraping [Nature Reviews Cancer](https://www.nature.com/nrc/)

This journal had no 'Open Access' articles at the time of data collection.

### Scraping [New England Journal of Medicine](https://www.nejm.org/)

#### Capturing a few document pages for inspection

In [3]:
url_sample_1 = 'https://www.nejm.org/toc/nejm/387/22'
url_sample_11 = 'https://www.nejm.org/doi/full/10.1056/NEJMoa2204783'

In [4]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/new_england_journal_of_medicine_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Closing the WebDriver
driver.quit()

In [5]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/new_england_journal_of_medicine_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [6]:
def scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url}{i}"
        driver.get(url)

        # Explicit wait for main article elements
        wait = WebDriverWait(driver, 10)

        # Ensure stable page load
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)

        # Parse page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # Extract Volume/Issue
        volume_issue = f"{volume}, Issue {i}"

        # Extract articles
        articles = soup.find_all('li', class_='toc_list-item')
        
        for article in articles:
            # Extract article type (if applicable)
            article_type_tag = article.find('input', class_='inputContentType')
            article_type = article_type_tag['value'] if article_type_tag else ''
        
            # Extract title
            title_tag = article.find('input', class_='inputArticleTitle')
            title = title_tag['value'] if title_tag else ''
        
            # Extract article URL
            doi_tag = article.find('input', class_='inputDoi')
            article_url = f"{root_url}/doi/full/{doi_tag['value']}" if doi_tag else ''
        
            # Extract authors
            authors_tag = article.find('input', class_='inputAuthor')
            authors = authors_tag['value'] if authors_tag else ''

            # Extract published date
            published_tag = article.find('input', class_='inputEPubDate')
            published = published_tag['value'] if published_tag else ''
        
            # Extract DOI
            doi = f"{doi_root_url}{doi_tag['value']}" if doi_tag else ''
        
            # Extract Free Access status
            free_access_tag = article.find('span', class_='issue-item_free')
            free_access = free_access_tag.get_text(strip=True) if free_access_tag else ''
        
            # Extract PDF URL
            pdf_url = f"{root_url}/doi/pdf/{doi_tag['value']}" if doi_tag else ''
        
            # Append extracted data
            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': published,
                'DOI': doi,
                'Free Access': free_access,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 382 (Issue 1 - 26)

NEJM's website blocks iterations by pushing a `CAPTCHA`. This requires scraping page by page.

In [7]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [8]:
start_page = 1
end_page = 1

In [9]:
df_v382i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.28s/it]


In [10]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [11]:
start_page = 2
end_page = 2

In [12]:
df_v382i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.43s/it]


In [13]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [14]:
start_page = 3
end_page = 3

In [15]:
df_v382i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.54s/it]


In [16]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [17]:
start_page = 4
end_page = 4

In [18]:
df_v382i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.37s/it]


In [19]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [20]:
start_page = 5
end_page = 5

In [21]:
df_v382i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.43s/it]


In [22]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [23]:
start_page = 6
end_page = 6

In [24]:
df_v382i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:10<00:00, 10.75s/it]


In [25]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [26]:
start_page = 7
end_page = 7

In [27]:
df_v382i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.54s/it]


In [28]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [29]:
start_page = 8
end_page = 8

In [30]:
df_v382i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.91s/it]


In [31]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [32]:
start_page = 9
end_page = 9

In [33]:
df_v382i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.46s/it]


In [34]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [35]:
start_page = 10
end_page = 10

In [36]:
df_v382i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.34s/it]


In [37]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [38]:
start_page = 11
end_page = 11

In [39]:
df_v382i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.94s/it]


In [40]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [41]:
start_page = 12
end_page = 12

In [42]:
df_v382i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.51s/it]


In [43]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [44]:
start_page = 13
end_page = 13

In [45]:
df_v382i13 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:11<00:00, 11.51s/it]


In [46]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [47]:
start_page = 14
end_page = 14

In [48]:
df_v382i14 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.30s/it]


In [49]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [50]:
start_page = 15
end_page = 15

In [51]:
df_v382i15 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.34s/it]


In [52]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [53]:
start_page = 16
end_page = 16

In [54]:
df_v382i16 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.53s/it]


In [55]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [56]:
start_page = 17
end_page = 17

In [57]:
df_v382i17 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.72s/it]


In [58]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [59]:
start_page = 18
end_page = 18

In [60]:
df_v382i18 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.26s/it]


In [61]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [62]:
start_page = 19
end_page = 19

In [63]:
df_v382i19 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.31s/it]


In [64]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [65]:
start_page = 20
end_page = 20

In [66]:
df_v382i20 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.62s/it]


In [67]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [68]:
start_page = 21
end_page = 21

In [69]:
df_v382i21 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.59s/it]


In [70]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [71]:
start_page = 22
end_page = 22

In [72]:
df_v382i22 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.48s/it]


In [73]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [74]:
start_page = 23
end_page = 23

In [75]:
df_v382i23 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.31s/it]


In [76]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [77]:
start_page = 24
end_page = 24

In [78]:
df_v382i24 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.35s/it]


In [79]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [80]:
start_page = 25
end_page = 25

In [81]:
df_v382i25 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.52s/it]


In [82]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/382/'
volume = 'Volume 382'
area_of_knowledge = 'Health Sciences'

In [83]:
start_page = 26
end_page = 26

In [84]:
df_v382i26 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.48s/it]


##### Volume 383 (Issue 1 - 27)

In [85]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [86]:
start_page = 1
end_page = 1

In [87]:
df_v383i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.47s/it]


In [88]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [89]:
start_page = 2
end_page = 2

In [90]:
df_v383i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.35s/it]


In [91]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [92]:
start_page = 3
end_page = 3

In [93]:
df_v383i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.41s/it]


In [94]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [95]:
start_page = 4
end_page = 4

In [96]:
df_v383i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.29s/it]


In [97]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [98]:
start_page = 5
end_page = 5

In [99]:
df_v383i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.45s/it]


In [100]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [101]:
start_page = 6
end_page = 6

In [102]:
df_v383i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.39s/it]


In [103]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [104]:
start_page = 7
end_page = 7

In [105]:
df_v383i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.55s/it]


In [106]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [107]:
start_page = 8
end_page = 8

In [108]:
df_v383i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.69s/it]


In [109]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [110]:
start_page = 9
end_page = 9

In [111]:
df_v383i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.35s/it]


In [112]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [113]:
start_page = 10
end_page = 10

In [114]:
df_v383i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.54s/it]


In [115]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [116]:
start_page = 11
end_page = 11

In [117]:
df_v383i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.90s/it]


In [118]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [119]:
start_page = 12
end_page = 12

In [120]:
df_v383i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.42s/it]


In [121]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [122]:
start_page = 13
end_page = 13

In [123]:
df_v383i13 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.42s/it]


In [124]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [125]:
start_page = 14
end_page = 14

In [126]:
df_v383i14 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.44s/it]


In [127]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [128]:
start_page = 15
end_page = 15

In [129]:
df_v383i15 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.30s/it]


In [130]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [131]:
start_page = 16
end_page = 16

In [132]:
df_v383i16 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.27s/it]


In [133]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [134]:
start_page = 17
end_page = 17

In [135]:
df_v383i17 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.43s/it]


In [136]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [137]:
start_page = 18
end_page = 18

In [138]:
df_v383i18 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.36s/it]


In [139]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [140]:
start_page = 19
end_page = 19

In [141]:
df_v383i19 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.74s/it]


In [142]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [143]:
start_page = 20
end_page = 20

In [144]:
df_v383i20 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.23s/it]


In [145]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [146]:
start_page = 21
end_page = 21

In [147]:
df_v383i21 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:11<00:00, 11.58s/it]


In [148]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [149]:
start_page = 22
end_page = 22

In [150]:
df_v383i22 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.31s/it]


In [151]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [152]:
start_page = 23
end_page = 23

In [153]:
df_v383i23 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.31s/it]


In [154]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [155]:
start_page = 24
end_page = 24

In [156]:
df_v383i24 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.38s/it]


In [157]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [158]:
start_page = 25
end_page = 25

In [159]:
df_v383i25 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.49s/it]


In [160]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [161]:
start_page = 26
end_page = 26

In [162]:
df_v383i26 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.39s/it]


In [163]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/383/'
volume = 'Volume 383'
area_of_knowledge = 'Health Sciences'

In [164]:
start_page = 27
end_page = 27

In [165]:
df_v383i27 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.40s/it]


##### Volume 384 (Issue 1 - 25)

In [166]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [167]:
start_page = 1
end_page = 1

In [168]:
df_v384i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.72s/it]


In [169]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [170]:
start_page = 2
end_page = 2

In [171]:
df_v384i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.41s/it]


In [172]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [173]:
start_page = 3
end_page = 3

In [174]:
df_v384i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.43s/it]


In [175]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [176]:
start_page = 4
end_page = 4

In [177]:
df_v384i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.79s/it]


In [178]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [179]:
start_page = 5
end_page = 5

In [180]:
df_v384i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.36s/it]


In [181]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [182]:
start_page = 6
end_page = 6

In [183]:
df_v384i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.35s/it]


In [184]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [185]:
start_page = 7
end_page = 7

In [186]:
df_v384i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.28s/it]


In [187]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [188]:
start_page = 8
end_page = 8

In [189]:
df_v384i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.33s/it]


In [190]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [191]:
start_page = 9
end_page = 9

In [192]:
df_v384i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.26s/it]


In [193]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [194]:
start_page = 10
end_page = 10

In [195]:
df_v384i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.35s/it]


In [196]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [197]:
start_page = 11
end_page = 11

In [198]:
df_v384i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.37s/it]


In [199]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [200]:
start_page = 12
end_page = 12

In [201]:
df_v384i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.49s/it]


In [202]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [203]:
start_page = 13
end_page = 13

In [204]:
df_v384i13 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.22s/it]


In [205]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [206]:
start_page = 14
end_page = 14

In [207]:
df_v384i14 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.26s/it]


In [208]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [209]:
start_page = 15
end_page = 15

In [210]:
df_v384i15 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.75s/it]


In [211]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [212]:
start_page = 16
end_page = 16

In [213]:
df_v384i16 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.15s/it]


In [214]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [215]:
start_page = 17
end_page = 17

In [216]:
df_v384i17 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.41s/it]


In [217]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [218]:
start_page = 18
end_page = 18

In [219]:
df_v384i18 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:12<00:00, 12.31s/it]


In [220]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [221]:
start_page = 19
end_page = 19

In [222]:
df_v384i19 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.08s/it]


In [223]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [224]:
start_page = 20
end_page = 20

In [225]:
df_v384i20 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.28s/it]


In [226]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [227]:
start_page = 21
end_page = 21

In [228]:
df_v384i21 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.29s/it]


In [229]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [230]:
start_page = 22
end_page = 22

In [231]:
df_v384i22 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.64s/it]


In [232]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [233]:
start_page = 23
end_page = 23

In [234]:
df_v384i23 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.47s/it]


In [235]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [236]:
start_page = 24
end_page = 24

In [237]:
df_v384i24 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.97s/it]


In [238]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/384/'
volume = 'Volume 384'
area_of_knowledge = 'Health Sciences'

In [239]:
start_page = 25
end_page = 25

In [240]:
df_v384i25 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.38s/it]


##### Volume 385 (Issue 1 - 27)

In [241]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [242]:
start_page = 1
end_page = 1

In [243]:
df_v385i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.47s/it]


In [244]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [245]:
start_page = 2
end_page = 2

In [246]:
df_v385i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.39s/it]


In [247]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [248]:
start_page = 3
end_page = 3

In [249]:
df_v385i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.34s/it]


In [250]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [251]:
start_page = 4
end_page = 4

In [252]:
df_v385i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.39s/it]


In [253]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [254]:
start_page = 5
end_page = 5

In [255]:
df_v385i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.89s/it]


In [256]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [257]:
start_page = 6
end_page = 6

In [258]:
df_v385i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.47s/it]


In [259]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [260]:
start_page = 7
end_page = 7

In [261]:
df_v385i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.21s/it]


In [262]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [263]:
start_page = 8
end_page = 8

In [264]:
df_v385i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.47s/it]


In [265]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [266]:
start_page = 9
end_page = 9

In [267]:
df_v385i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.49s/it]


In [268]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [269]:
start_page = 10
end_page = 10

In [270]:
df_v385i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.35s/it]


In [271]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [272]:
start_page = 11
end_page = 11

In [273]:
df_v385i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.33s/it]


In [274]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [275]:
start_page = 12
end_page = 12

In [276]:
df_v385i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.66s/it]


In [277]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [278]:
start_page = 13
end_page = 13

In [279]:
df_v385i13 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.24s/it]


In [280]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [281]:
start_page = 14
end_page = 14

In [282]:
df_v385i14 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.47s/it]


In [283]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [284]:
start_page = 15
end_page = 15

In [285]:
df_v385i15 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.50s/it]


In [286]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [287]:
start_page = 16
end_page = 16

In [288]:
df_v385i16 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.48s/it]


In [289]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [290]:
start_page = 17
end_page = 17

In [291]:
df_v385i17 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.36s/it]


In [292]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [293]:
start_page = 18
end_page = 18

In [294]:
df_v385i18 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.30s/it]


In [295]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [296]:
start_page = 19
end_page = 19

In [297]:
df_v385i19 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.17s/it]


In [298]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [299]:
start_page = 20
end_page = 20

In [300]:
df_v385i20 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.22s/it]


In [301]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [302]:
start_page = 21
end_page = 21

In [303]:
df_v385i21 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.33s/it]


In [304]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [305]:
start_page = 22
end_page = 22

In [306]:
df_v385i22 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.26s/it]


In [307]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [308]:
start_page = 23
end_page = 23

In [309]:
df_v385i23 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.51s/it]


In [310]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [311]:
start_page = 24
end_page = 24

In [312]:
df_v385i24 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.21s/it]


In [313]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [314]:
start_page = 25
end_page = 25

In [315]:
df_v385i25 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.27s/it]


In [316]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [317]:
start_page = 26
end_page = 26

In [318]:
df_v385i26 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.45s/it]


In [319]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/385/'
volume = 'Volume 385'
area_of_knowledge = 'Health Sciences'

In [320]:
start_page = 27
end_page = 27

In [321]:
df_v385i27 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.94s/it]


##### Volume 386 (Issue 1 - 26)

In [322]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [323]:
start_page = 1
end_page = 1

In [324]:
df_v386i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.45s/it]


In [325]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [326]:
start_page = 2
end_page = 2

In [327]:
df_v386i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.38s/it]


In [328]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [329]:
start_page = 3
end_page = 3

In [330]:
df_v386i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.18s/it]


In [331]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [332]:
start_page = 4
end_page = 4

In [333]:
df_v386i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.34s/it]


In [334]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [335]:
start_page = 5
end_page = 5

In [336]:
df_v386i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.14s/it]


In [337]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [338]:
start_page = 6
end_page = 6

In [339]:
df_v386i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.30s/it]


In [340]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [341]:
start_page = 7
end_page = 7

In [342]:
df_v386i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.62s/it]


In [343]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [344]:
start_page = 8
end_page = 8

In [345]:
df_v386i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.17s/it]


In [346]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [347]:
start_page = 9
end_page = 9

In [348]:
df_v386i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.27s/it]


In [349]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [350]:
start_page = 10
end_page = 10

In [351]:
df_v386i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.43s/it]


In [352]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [353]:
start_page = 11
end_page = 11

In [354]:
df_v386i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.38s/it]


In [355]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [356]:
start_page = 12
end_page = 12

In [357]:
df_v386i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.48s/it]


In [358]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [359]:
start_page = 13
end_page = 13

In [360]:
df_v386i13 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.18s/it]


In [361]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [362]:
start_page = 14
end_page = 14

In [363]:
df_v386i14 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:07<00:00,  7.34s/it]


In [364]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [365]:
start_page = 15
end_page = 15

In [366]:
df_v386i15 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.37s/it]


In [367]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [368]:
start_page = 16
end_page = 16

In [369]:
df_v386i16 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.74s/it]


In [370]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [371]:
start_page = 17
end_page = 17

In [372]:
df_v386i17 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.26s/it]


In [373]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [374]:
start_page = 18
end_page = 18

In [375]:
df_v386i18 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.29s/it]


In [376]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [377]:
start_page = 19
end_page = 19

In [378]:
df_v386i19 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.58s/it]


In [379]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [380]:
start_page = 20
end_page = 20

In [381]:
df_v386i20 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.47s/it]


In [382]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [383]:
start_page = 21
end_page = 21

In [384]:
df_v386i21 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.32s/it]


In [385]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [386]:
start_page = 22
end_page = 22

In [387]:
df_v386i22 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.32s/it]


In [388]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [389]:
start_page = 23
end_page = 23

In [390]:
df_v386i23 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.47s/it]


In [391]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [392]:
start_page = 24
end_page = 24

In [393]:
df_v386i24 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.32s/it]


In [394]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [395]:
start_page = 25
end_page = 25

In [396]:
df_v386i25 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.39s/it]


In [397]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/386/'
volume = 'Volume 386'
area_of_knowledge = 'Health Sciences'

In [398]:
start_page = 26
end_page = 26

In [399]:
df_v386i26 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.43s/it]


##### Volume 387 (Issue 1 - 22)

In [400]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [401]:
start_page = 1
end_page = 1

In [402]:
df_v387i1 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.52s/it]


In [403]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [404]:
start_page = 2
end_page = 2

In [405]:
df_v387i2 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.17s/it]


In [406]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [407]:
start_page = 3
end_page = 3

In [408]:
df_v387i3 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.28s/it]


In [409]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [410]:
start_page = 4
end_page = 4

In [411]:
df_v387i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.61s/it]


In [412]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [413]:
start_page = 5
end_page = 5

In [414]:
df_v387i5 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.62s/it]


In [415]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [416]:
start_page = 6
end_page = 6

In [417]:
df_v387i6 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.34s/it]


In [418]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [419]:
start_page = 7
end_page = 7

In [420]:
df_v387i7 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.24s/it]


In [421]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [422]:
start_page = 8
end_page = 8

In [423]:
df_v387i8 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.49s/it]


In [424]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [425]:
start_page = 9
end_page = 9

In [426]:
df_v387i9 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.05s/it]


In [427]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [428]:
start_page = 10
end_page = 10

In [429]:
df_v387i10 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.40s/it]


In [430]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [431]:
start_page = 11
end_page = 11

In [432]:
df_v387i11 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.45s/it]


In [433]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [434]:
start_page = 12
end_page = 12

In [435]:
df_v387i12 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.45s/it]


In [436]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [437]:
start_page = 13
end_page = 13

In [438]:
df_v387i13 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.50s/it]


In [439]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [440]:
start_page = 14
end_page = 14

In [441]:
df_v387i14 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.99s/it]


In [442]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [443]:
start_page = 15
end_page = 15

In [444]:
df_v387i15 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:09<00:00,  9.32s/it]


In [445]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [446]:
start_page = 16
end_page = 16

In [447]:
df_v387i16 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.44s/it]


In [448]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [449]:
start_page = 17
end_page = 17

In [450]:
df_v387i17 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.42s/it]


In [451]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [452]:
start_page = 18
end_page = 18

In [453]:
df_v387i18 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.39s/it]


In [454]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [455]:
start_page = 19
end_page = 19

In [456]:
df_v387i19 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.40s/it]


In [457]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [458]:
start_page = 20
end_page = 20

In [459]:
df_v387i20 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.32s/it]


In [460]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [461]:
start_page = 21
end_page = 21

In [462]:
df_v387i21 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:08<00:00,  8.98s/it]


In [463]:
root_url = 'https://www.nejm.org'
doi_root_url = 'https://doi.org'
article_list_url = 'https://www.nejm.org/toc/nejm/387/'
volume = 'Volume 387'
area_of_knowledge = 'Health Sciences'

In [464]:
start_page = 22
end_page = 22

In [465]:
df_v387i22 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, volume, area_of_knowledge)

100%|██████████| 1/1 [00:06<00:00,  6.50s/it]


##### Concatenating the DataFrames

In [488]:
df_new_england_journal_of_medicine = pd.concat([
    df_v382i1, df_v382i2, df_v382i3, df_v382i4, df_v382i5, df_v382i6, df_v382i7, df_v382i8, df_v382i9, df_v382i10, df_v382i11, df_v382i12, df_v382i13, df_v382i14, df_v382i15, df_v382i16, df_v382i17, df_v382i18, df_v382i19, df_v382i20, df_v382i21, df_v382i22, df_v382i23, df_v382i24, df_v382i25, df_v382i26,
    df_v383i1, df_v383i2, df_v383i3, df_v383i4, df_v383i5, df_v383i6, df_v383i7, df_v383i8, df_v383i9, df_v383i10, df_v383i11, df_v383i12, df_v383i13, df_v383i14, df_v383i15, df_v383i16, df_v383i17, df_v383i18, df_v383i19, df_v383i20, df_v383i21, df_v383i22, df_v383i23, df_v383i24, df_v383i25, df_v383i26, df_v383i27,
    df_v384i1, df_v384i2, df_v384i3, df_v384i4, df_v384i5, df_v384i6, df_v384i7, df_v384i8, df_v384i9, df_v384i10, df_v384i11, df_v384i12, df_v384i13, df_v384i14, df_v384i15, df_v384i16, df_v384i17, df_v384i18, df_v384i19, df_v384i20, df_v384i21, df_v384i22, df_v384i23, df_v384i24, df_v384i25,
    df_v385i1, df_v385i2, df_v385i3, df_v385i4, df_v385i5, df_v385i6, df_v385i7, df_v385i8, df_v385i9, df_v385i10, df_v385i11, df_v385i12, df_v385i13, df_v385i14, df_v385i15, df_v385i16, df_v385i17, df_v385i18, df_v385i19, df_v385i20, df_v385i21, df_v385i22, df_v385i23, df_v385i24, df_v385i25, df_v385i26, df_v385i27,
    df_v386i1, df_v386i2, df_v386i3, df_v386i4, df_v386i5, df_v386i6, df_v386i7, df_v386i8, df_v386i9, df_v386i10, df_v386i11, df_v386i12, df_v386i13, df_v386i14, df_v386i15, df_v386i16, df_v386i17, df_v386i18, df_v386i19, df_v386i20, df_v386i21, df_v386i22, df_v386i23, df_v386i24, df_v386i25, df_v386i26,
    df_v387i1, df_v387i2, df_v387i3, df_v387i4, df_v387i5, df_v387i6, df_v387i7, df_v387i8, df_v387i9, df_v387i10, df_v387i11, df_v387i12, df_v387i13, df_v387i14, df_v387i15, df_v387i16, df_v387i17, df_v387i18, df_v387i19, df_v387i20, df_v387i21, df_v387i22
], ignore_index=True)

In [489]:
df_new_england_journal_of_medicine['Published'] = pd.to_datetime(
    df_new_england_journal_of_medicine['Published'], format='mixed', errors='coerce'
)

In [490]:
df_new_england_journal_of_medicine.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Free Access                  object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [491]:
unique_open_access = df_new_england_journal_of_medicine['Free Access'].dropna().unique()
unique_open_access

array(['FREE', ''], dtype=object)

In [492]:
df_new_england_journal_of_medicine_open_access = df_new_england_journal_of_medicine[
    (df_new_england_journal_of_medicine['Free Access'] == 'FREE')
]
df_new_england_journal_of_medicine_open_access = df_new_england_journal_of_medicine_open_access[df_new_england_journal_of_medicine_open_access['Published'] <= '2022-11-30']
df_new_england_journal_of_medicine_open_access = df_new_england_journal_of_medicine_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [493]:
unique_article_type = df_new_england_journal_of_medicine_open_access['Article Type'].dropna().unique()
unique_article_type

array(['Perspective', 'Original Articles', 'Special Article',
       'Editorials', 'Correspondence', 'Correction', 'Corrections',
       'Special Report', 'Review Article', 'Sounding Board',
       'Videos in Clinical Medicine',
       'Case Records of the Massachusetts General Hospital',
       'Clinical Implications of Basic Research', 'Medicine and Society',
       'Points of View', 'Images in Clinical Medicine',
       'Clinical Decisions', 'Clinical Practice',
       'Interactive Medical Case', 'Clinical Problem-Solving'],
      dtype=object)

In [494]:
df_new_england_journal_of_medicine_open_access = df_new_england_journal_of_medicine_open_access.loc[
    df_new_england_journal_of_medicine_open_access['Article Type'].isin(['Original Articles', 'Special Article', 'Review Article'])
].reset_index(drop=True)

In [495]:
df_new_england_journal_of_medicine_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Free Access,PDF URL,Area of Knowledge
0,Original Articles,A Comparison of Two LDL Cholesterol Targets af...,https://www.nejm.org/doi/full/10.1056/NEJMoa19...,P. Amarenco and Others,"Volume 382, Issue 1",2019-11-18,https://doi.org10.1056/NEJMoa1910355,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa191...,Health Sciences
1,Original Articles,Alcohol Abstinence in Drinkers with Atrial Fib...,https://www.nejm.org/doi/full/10.1056/NEJMoa18...,A. Voskoboinik and Others,"Volume 382, Issue 1",2020-01-01,https://doi.org10.1056/NEJMoa1817591,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa181...,Health Sciences
2,Original Articles,Multiyear Follow-up of AAV5-hFVIII-SQ Gene The...,https://www.nejm.org/doi/full/10.1056/NEJMoa19...,K.J. Pasi and Others,"Volume 382, Issue 1",2020-01-01,https://doi.org10.1056/NEJMoa1908490,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa190...,Health Sciences
3,Original Articles,Overall Survival with Osimertinib in Untreated...,https://www.nejm.org/doi/full/10.1056/NEJMoa19...,S.S. Ramalingam and Others,"Volume 382, Issue 1",2019-11-21,https://doi.org10.1056/NEJMoa1913662,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa191...,Health Sciences
4,Special Article,Changes in Quality of Care after Hospital Merg...,https://www.nejm.org/doi/full/10.1056/NEJMsa19...,N.D. Beaulieu and Others,"Volume 382, Issue 1",2020-01-01,https://doi.org10.1056/NEJMsa1901383,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMsa190...,Health Sciences
...,...,...,...,...,...,...,...,...,...,...
662,Review Article,Fossil-Fuel Pollution and Climate Change:\n ...,https://www.nejm.org/doi/full/10.1056/NEJMra22...,M.C. Thomson and L.R. Stanberry,"Volume 387, Issue 21",2022-11-23,https://doi.org10.1056/NEJMra2200092,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMra220...,Health Sciences
663,Original Articles,Renin–Angiotensin System Inhibition in Advance...,https://www.nejm.org/doi/full/10.1056/NEJMoa22...,S. Bhandari and Others,"Volume 387, Issue 22",2022-11-03,https://doi.org10.1056/NEJMoa2210639,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa221...,Health Sciences
664,Original Articles,Buprenorphine versus Methadone for Opioid Use ...,https://www.nejm.org/doi/full/10.1056/NEJMoa22...,E.A. Suarez and Others,"Volume 387, Issue 22",2022-11-30,https://doi.org10.1056/NEJMoa2203318,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa220...,Health Sciences
665,Original Articles,Trial of Deferiprone in Parkinson’s Disease,https://www.nejm.org/doi/full/10.1056/NEJMoa22...,D. Devos and Others,"Volume 387, Issue 22",2022-11-30,https://doi.org10.1056/NEJMoa2209254,FREE,https://www.nejm.org/doi/pdf/10.1056/NEJMoa220...,Health Sciences


#### Inspecting a few articles

In [496]:
df_new_england_journal_of_medicine_open_access.at[4, 'URL']

'https://www.nejm.org/doi/full/10.1056/NEJMsa1901383'

#### Exporting to a file

In [497]:
df_new_england_journal_of_medicine_open_access.to_json(f"{output_directory}/new_england_journal_of_medicine_open_access.jsonl", orient='records', lines=True)

### Scraping [The Lancet](https://www.thelancet.com/journals/lancet/home)

This journal had no 'Open Access' articles at the time of data collection.

## Human Sciences

### Scraping [Annual Review of Anthropology](https://www.annualreviews.org/content/journals/anthro)

#### Scraping the articles

##### Function to scrape the articles

Make sure you run the function `scrape_articles` on section `Scraping Annual Review of Plant Biology` before scraping the articles.

##### Volume 51, 2022

In [33]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/anthro/51/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Human Sciences'

In [34]:
df_2022 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:06<00:00,  3.32s/it]


##### Volume 50, 2021

In [35]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/anthro/50/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Human Sciences'

In [36]:
df_2021 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:07<00:00,  3.77s/it]


##### Volume 49, 2020

In [37]:
root_url = 'https://www.annualreviews.org'
article_list_url = 'https://www.annualreviews.org/content/journals/anthro/49/1?page='
start_page = 1
end_page = 2
area_of_knowledge = 'Human Sciences'

In [38]:
df_2020 = scrape_articles(article_list_url, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 2/2 [00:07<00:00,  3.70s/it]


##### Concatenating the DataFrames

In [39]:
df_ar_anthropology = pd.concat([df_2022, df_2021, df_2020], ignore_index=True)
df_ar_anthropology

,Title,URL,Authors,Vol/Year/Page Range,DOI,Area of Knowledge
0,Communities of Thought,https://www.annualreviews.org/content/journals...,"Don Brenneis, , andKaren B. Strier","Vol. 51 \n(2022),pp.v–vi",https://doi.org/10.1146/annurev-an-51-082222-1...,Human Sciences
1,Thinking in Between Disciplines,https://www.annualreviews.org/content/journals...,Elinor Ochs,"Vol. 51 \n(2022),pp.1–15",https://doi.org/10.1146/annurev-anthro-070120-...,Human Sciences
2,Bad Mouths: Taboo and Transgressive Language,https://www.annualreviews.org/content/journals...,Laura Miller,"Vol. 51 \n(2022),pp.17–30",https://doi.org/10.1146/annurev-anthro-041420-...,Human Sciences
3,The Necropolitics of Language Oppression,https://www.annualreviews.org/content/journals...,Gerald Roche,"Vol. 51 \n(2022),pp.31–47",https://doi.org/10.1146/annurev-anthro-041420-...,Human Sciences
4,The Carceral State: An American Story,https://www.annualreviews.org/content/journals...,Aisha Khan,"Vol. 51 \n(2022),pp.49–66",https://doi.org/10.1146/annurev-anthro-041420-...,Human Sciences
...,...,...,...,...,...,...
76,Madness: Recursive Ethnography and the Critica...,https://www.annualreviews.org/content/journals...,Sarah Pinto,"Vol. 49 \n(2020),pp.299–316",https://doi.org/10.1146/annurev-anthro-010220-...,Human Sciences
77,Socio-Ecological Challenges as Modulators of W...,https://www.annualreviews.org/content/journals...,"Pablo A. Nepomnaschy, ,Amanda Rowlands, ,Ana P...","Vol. 49 \n(2020),pp.317–336",https://doi.org/10.1146/annurev-anthro-102317-...,Human Sciences
78,Archaeology of Cuisine and Cooking,https://www.annualreviews.org/content/journals...,Sarah R. Graff,"Vol. 49 \n(2020),pp.337–354",https://doi.org/10.1146/annurev-anthro-102317-...,Human Sciences
79,Beyond the Household: Caribbean Families and B...,https://www.annualreviews.org/content/journals...,Robin G. Nelson,"Vol. 49 \n(2020),pp.355–372",https://doi.org/10.1146/annurev-anthro-102218-...,Human Sciences


#### Inspecting a few articles

**Attention**: Some of the documents might in fact be `Editorials` rather than `Research articles`. Extra refinement is recommended when using the dataset.

In [40]:
df_ar_anthropology.at[0, 'URL']

'https://www.annualreviews.org/content/journals/10.1146/annurev-an-51-082222-100001'

#### Exporting to a file

In [41]:
df_ar_anthropology.to_json(f"{output_directory}/ar_anthropology.jsonl", orient='records', lines=True)

### Scraping [Journal of Human Evolution](https://www.sciencedirect.com/journal/journal-of-human-evolution)

#### Capturing a few document pages for inspection

In [3]:
url_sample_1 = 'https://www.sciencedirect.com/journal/journal-of-human-evolution/vol/138/suppl/C'
url_sample_11 = 'https://www.sciencedirect.com/science/article/pii/S0047248419303124'

In [6]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/journal_human_evolution_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/journal_human_evolution_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [33]:
def scrape_articles(article_list_url1, article_list_url2, root_url, start_page, end_page, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url1}{i}{article_list_url2}"
        driver.get(url)
        wait = WebDriverWait(driver, 10)
        #wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')

        volume_issue_tag = soup.find('h2', class_='u-text-light js-vol-issue')
        volume_issue = volume_issue_tag.get_text(strip=True) if volume_issue_tag else ''

        month_year_tag = soup.find('h3', class_='js-issue-status text-s')
        month_year = month_year_tag.get_text(strip=True) if month_year_tag else ''

        articles = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')

        for article in articles:
            article_type_tag = article.find('span', class_='js-article-subtype')
            article_type = article_type_tag.get_text(strip=True) if article_type_tag else ''

            title_tag = article.find('span', class_='js-article-title text-l')
            title = title_tag.get_text(strip=True) if title_tag else ''

            article_url_tag = article.find('h3', class_='text-m u-font-serif u-display-inline').find('a')
            article_url = root_url + article_url_tag['href'] if article_url_tag else ''

            authors_tags = article.find('div', class_='text-s u-clr-grey8 js-article__item__authors')
            authors = authors_tags.get_text(strip=True) if authors_tags else ''

            doi_tag = article.find('div', attrs={'hidden': True})  # Explicitly target the 'hidden' attribute
            doi = doi_tag.get_text(strip=True) if doi_tag else ''

            open_access_tag = article.find('span', class_='js-open-access u-text-italic')
            open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

            open_access1_tag = article.find('span', class_='js-open-access')
            open_access1 = open_access1_tag.get_text(strip=True) if open_access1_tag else ''

            pdf_url_tag = article.find('dd', class_='js-article-link u-margin-xs-top u-margin-s-bottom article-item-link')
            pdf_url = root_url + pdf_url_tag.find('a')['href'] if pdf_url_tag and pdf_url_tag.find('a') else ''

            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': month_year,
                'DOI': doi,
                'Open Access': open_access,
                'Open Access 1': open_access1,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    driver.quit()

    return pd.DataFrame(data)

##### Volumes 138 to 172

In [34]:
root_url = 'https://www.sciencedirect.com/'
article_list_url1 = 'https://www.sciencedirect.com/journal/journal-of-human-evolution/vol/'
article_list_url2 = '/suppl/C'
start_page = 138
end_page = 172
area_of_knowledge = 'Human Sciences'

In [35]:
df_journal_human_evolution = scrape_articles(article_list_url1, article_list_url2, root_url, start_page, end_page, area_of_knowledge)

100%|██████████| 35/35 [03:29<00:00,  6.00s/it]


In [36]:
df_journal_human_evolution['Published'] = pd.to_datetime(
    df_journal_human_evolution['Published'], format='%B %Y'
)

In [37]:
df_journal_human_evolution.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
Open Access 1                object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [38]:
unique_open_access = df_journal_human_evolution['Open Access'].dropna().unique()
unique_open_access

array(['', 'Open access'], dtype=object)

In [39]:
unique_open_access1 = df_journal_human_evolution['Open Access 1'].dropna().unique()
unique_open_access1

array(['Full text access', 'Abstract only', 'No access', 'Open access'],
      dtype=object)

In [40]:
df_journal_human_evolution_open_access = df_journal_human_evolution[
    (df_journal_human_evolution['Open Access'] == 'Open access') | 
    (df_journal_human_evolution['Open Access 1'] == 'Full text access') | 
    (df_journal_human_evolution['Open Access 1'] == 'Open access')
]
df_journal_human_evolution_open_access = df_journal_human_evolution_open_access[df_journal_human_evolution_open_access['Published'] <= '2022-11-30']
df_journal_human_evolution_open_access = df_journal_human_evolution_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [41]:
df_journal_human_evolution_open_access = df_journal_human_evolution_open_access.loc[
    df_journal_human_evolution_open_access['Article Type'] == 'Research article'
].reset_index(drop=True)
df_journal_human_evolution_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,Open Access 1,PDF URL,Area of Knowledge
0,Research article,Increased terrestriality in a Neotropical prim...,https://www.sciencedirect.com//science/article...,"Claudio M. Monteza-Moreno, Margaret C. Crofoot...",Volume 143,2020-06-01,https://doi.org/10.1016/j.jhevol.2020.102768,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
1,Research article,"Mesopithecus pentelicusfrom Zhaotong, China, t...",https://www.sciencedirect.com//science/article...,"Nina G. Jablonski, Xueping Ji, Jay Kelley, Law...",Volume 146,2020-09-01,https://doi.org/10.1016/j.jhevol.2020.102851,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
2,Research article,Mitogenomics of macaques (Macaca) across Walla...,https://www.sciencedirect.com//science/article...,"Ben J. Evans, Marie-Theres Gansauge, Matthew W...",Volume 146,2020-09-01,https://doi.org/10.1016/j.jhevol.2020.102852,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
3,Research article,Patterns of urinary cortisol levels during ont...,https://www.sciencedirect.com//science/article...,"Patrick J. Tkaczynski, Verena Behringer, Corin...",Volume 147,2020-10-01,https://doi.org/10.1016/j.jhevol.2020.102869,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
4,Research article,Early anthropogenic use of hematite on Aurigna...,https://www.sciencedirect.com//science/article...,"Elizabeth C. Velliky, Patrick Schmidt, Ludovic...",Volume 150,2021-01-01,https://doi.org/10.1016/j.jhevol.2020.102900,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
5,Research article,A West African Middle Stone Age site dated to ...,https://www.sciencedirect.com//science/article...,"Katja Douze, Laurent Lespez, Michel Rasse, Cha...",Volume 154,2021-05-01,https://doi.org/10.1016/j.jhevol.2021.102952,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
6,Research article,Dietary evidence from Central Asian Neandertha...,https://www.sciencedirect.com//science/article...,"Domingo C. Salazar-García, Robert C. Power, Na...",Volume 156,2021-07-01,https://doi.org/10.1016/j.jhevol.2021.102985,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
7,Research article,Dental microstructure records life history eve...,https://www.sciencedirect.com//science/article...,"Simone A.M. Lemmers, Wendy Dirks, Sally E. Str...",Volume 158,2021-09-01,https://doi.org/10.1016/j.jhevol.2021.103046,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
8,Research article,Evidence of habitual behavior from non-aliment...,https://www.sciencedirect.com//science/article...,"Almudena Estalrrich, Ana B. Marín-Arroyo",Volume 158,2021-09-01,https://doi.org/10.1016/j.jhevol.2021.103047,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences
9,Research article,Effects of hybridization on pelvic morphology:...,https://www.sciencedirect.com//science/article...,"Laura T. Buck, David C. Katz, Rebecca Rogers A...",Volume 159,2021-10-01,https://doi.org/10.1016/j.jhevol.2021.103049,Open access,Open access,https://www.sciencedirect.com//science/article...,Human Sciences


#### Inspecting a few articles

In [42]:
df_journal_human_evolution_open_access.at[0, 'URL']

'https://www.sciencedirect.com//science/article/pii/S0047248420300294'

#### Exporting to a file

In [43]:
df_journal_human_evolution_open_access.to_json(f"{output_directory}/journal_human_evolution_open_access.jsonl", orient='records', lines=True)

## Linguistics, literature and arts

### Scraping [Applied Corpus Linguistics](https://www.sciencedirect.com/journal/applied-corpus-linguistics)

#### Capturing a few document pages for inspection

In [3]:
url_sample_1 = 'https://www.sciencedirect.com/journal/applied-corpus-linguistics/vol/1/issue/1'
url_sample_11 = 'https://www.sciencedirect.com/science/article/pii/S2666799121000010'

In [13]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/applied_corpus_linguistics_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/applied_corpus_linguistics_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [32]:
def scrape_articles(article_list_url, root_url, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    driver.get(article_list_url)
    wait = WebDriverWait(driver, 10)
    #wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))
    max_wait_time = 30
    start_time = time.time()
    previous_html = ''
    while True:
        current_html = driver.page_source
        if current_html == previous_html or time.time() - start_time > max_wait_time:
            break
        previous_html = current_html
        time.sleep(2)
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
    
    volume_issue_tag = soup.find('h2', class_='u-text-light js-vol-issue')
    volume_issue = volume_issue_tag.get_text(strip=True) if volume_issue_tag else ''

    month_year_tag = soup.find('h3', class_='js-issue-status text-s')
    month_year = month_year_tag.get_text(strip=True) if month_year_tag else ''

    articles = soup.find_all('li', class_='js-article-list-item article-item u-padding-xs-top u-margin-l-bottom')

    for article in articles:
        article_type_tag = article.find('span', class_='js-article-subtype')
        article_type = article_type_tag.get_text(strip=True) if article_type_tag else ''
        
        title_tag = article.find('span', class_='js-article-title text-l')
        title = title_tag.get_text(strip=True) if title_tag else ''
    
        article_url_tag = article.find('h3', class_='text-m u-font-serif u-display-inline').find('a')
        article_url = root_url + article_url_tag['href'] if article_url_tag else ''
    
        authors_tags = article.find('div', class_='text-s u-clr-grey8 js-article__item__authors')
        authors = authors_tags.get_text(strip=True) if authors_tags else ''
    
        doi_tag = article.find('div', attrs={'hidden': True})  # Explicitly target the 'hidden' attribute
        doi = doi_tag.get_text(strip=True) if doi_tag else ''
    
        open_access_tag = article.find('span', class_='js-open-access u-text-italic')
        open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''
    
        pdf_url_tag = article.find('dd', class_='js-article-link u-margin-xs-top u-margin-s-bottom article-item-link')
        pdf_url = root_url + pdf_url_tag.find('a')['href'] if pdf_url_tag and pdf_url_tag.find('a') else ''
    
        data.append({
            'Article Type': article_type,
            'Title': title,
            'URL': article_url,
            'Authors': authors,
            'Vol/Issue': volume_issue,
            'Published': month_year,
            'DOI': doi,
            'Open Access': open_access,
            'PDF URL': pdf_url,
            'Area of Knowledge': area_of_knowledge
        })

    driver.quit()

    return pd.DataFrame(data)

##### Volume 1, Issue 1

In [4]:
root_url = 'https://www.sciencedirect.com/'
article_list_url = 'https://www.sciencedirect.com/journal/applied-corpus-linguistics/vol/1/issue/1'
area_of_knowledge = 'Linguistic, literature and arts'

In [5]:
df_v1i1 = scrape_articles(article_list_url, root_url, area_of_knowledge)

##### Volume 1, Issue 2

In [6]:
root_url = 'https://www.sciencedirect.com/'
article_list_url = 'https://www.sciencedirect.com/journal/applied-corpus-linguistics/vol/1/issue/2'
area_of_knowledge = 'Linguistic, literature and arts'

In [7]:
df_v1i2 = scrape_articles(article_list_url, root_url, area_of_knowledge)

##### Volume 1, Issue 3

In [8]:
root_url = 'https://www.sciencedirect.com/'
article_list_url = 'https://www.sciencedirect.com/journal/applied-corpus-linguistics/vol/1/issue/3'
area_of_knowledge = 'Linguistic, literature and arts'

In [9]:
df_v1i3 = scrape_articles(article_list_url, root_url, area_of_knowledge)

##### Volume 2, Issue 1

In [10]:
root_url = 'https://www.sciencedirect.com/'
article_list_url = 'https://www.sciencedirect.com/journal/applied-corpus-linguistics/vol/2/issue/1'
area_of_knowledge = 'Linguistic, literature and arts'

In [11]:
df_v2i1 = scrape_articles(article_list_url, root_url, area_of_knowledge)

##### Volume 2, Issue 2

In [12]:
root_url = 'https://www.sciencedirect.com/'
article_list_url = 'https://www.sciencedirect.com/journal/applied-corpus-linguistics/vol/2/issue/2'
area_of_knowledge = 'Linguistic, literature and arts'

In [13]:
df_v2i2 = scrape_articles(article_list_url, root_url, area_of_knowledge)

##### Concatenating the DataFrames

In [14]:
df_applied_corpus_linguistics = pd.concat([df_v1i1, df_v1i2, df_v1i3, df_v2i1, df_v2i2], ignore_index=True)

In [15]:
df_applied_corpus_linguistics['Published'] = pd.to_datetime(
    df_applied_corpus_linguistics['Published'], format='%B %Y'
)

In [16]:
df_applied_corpus_linguistics.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' articles published before the advent of ChatGPT

In [17]:
unique_open_access = df_applied_corpus_linguistics['Open Access'].dropna().unique()
unique_open_access

array(['Open access', ''], dtype=object)

In [18]:
df_applied_corpus_linguistics_open_access = df_applied_corpus_linguistics[df_applied_corpus_linguistics['Open Access'] == 'Open access']
df_applied_corpus_linguistics_open_access = df_applied_corpus_linguistics_open_access[df_applied_corpus_linguistics_open_access['Published'] <= '2022-11-30']
df_applied_corpus_linguistics_open_access = df_applied_corpus_linguistics_open_access.reset_index(drop=True)
df_applied_corpus_linguistics_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,PDF URL,Area of Knowledge
0,Research article,Investigating predatory publishing in politica...,https://www.sciencedirect.com//science/article...,"Ying Wang, Josep Soler","Volume 1, Issue 1",2021-04-01,https://doi.org/10.1016/j.acorp.2021.100001,Open access,https://www.sciencedirect.com//science/article...,"Linguistic, literature and arts"
1,Research article,The role of modals in policies: The US opioid ...,https://www.sciencedirect.com//science/article...,Peter Joseph Torres,"Volume 1, Issue 3",2021-12-01,https://doi.org/10.1016/j.acorp.2021.100008,Open access,https://www.sciencedirect.com//science/article...,"Linguistic, literature and arts"
2,Research article,Patient feedback and duration of treatment: A ...,https://www.sciencedirect.com//science/article...,"Gavin Brookes, Paul Baker","Volume 1, Issue 3",2021-12-01,https://doi.org/10.1016/j.acorp.2021.100010,Open access,https://www.sciencedirect.com//science/article...,"Linguistic, literature and arts"
3,Research article,Creating a corpus of sensitive and hard-to-acc...,https://www.sciencedirect.com//science/article...,"Maria Leedham, Theresa Lillis, Alison Twiner","Volume 1, Issue 3",2021-12-01,https://doi.org/10.1016/j.acorp.2021.100011,Open access,https://www.sciencedirect.com//science/article...,"Linguistic, literature and arts"
4,Research article,Compiling and analysing a large corpus of onli...,https://www.sciencedirect.com//science/article...,Shi Min CHUA,"Volume 2, Issue 2",2022-08-01,https://doi.org/10.1016/j.acorp.2022.100017,Open access,https://www.sciencedirect.com//science/article...,"Linguistic, literature and arts"


#### Inspecting a few articles

In [19]:
df_applied_corpus_linguistics_open_access.at[0, 'URL']

'https://www.sciencedirect.com//science/article/pii/S2666799121000010'

#### Exporting to a file

In [20]:
df_applied_corpus_linguistics_open_access.to_json(f"{output_directory}/applied_corpus_linguistics_open_access.jsonl", orient='records', lines=True)

### Scraping [Journal of Aesthetic Education](https://press-jae.scholasticahq.com/)

This journal was not open for the public at the time of data collection.

### Scraping [Journal of English Linguistics](https://journals.sagepub.com/home/eng)

#### Capturing a few document pages for inspection

In [27]:
url_sample_1 = 'https://journals.sagepub.com/toc/enga/49/3'
url_sample_11 = 'https://journals.sagepub.com/doi/full/10.1177/00754242211024722'

In [28]:
# Setting up the WebDriver (make sure you have downloaded the Microsoft Edge WebDriver executable)
# https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/
service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
driver = webdriver.Edge(service=service)

# Navigating to target URL 1 and saving its web page
driver.get(url_sample_1)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_1 = driver.page_source

with open(f'{output_directory}/journal_english_linguistics_sample1.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_1)

# Navigating to target URL 2 and saving its web page
driver.get(url_sample_11)
wait = WebDriverWait(driver, 10)
#time.sleep(15)  # Waits for 15 seconds
#wait.until(EC.presence_of_element_located((By.TAG_NAME, 'html')))

# Extra reliability check: Wait until the page source stops changing
max_wait_time = 30  # Max time in seconds
start_time = time.time()
previous_html = ''
while True:
    current_html = driver.page_source
    if current_html == previous_html or time.time() - start_time > max_wait_time:
        break  # Exit loop if page stops changing or max wait time is exceeded
    previous_html = current_html
    time.sleep(2)  # Short delay before checking again

# Now, the page is fully loaded - extract content!

document_page_sample_11 = driver.page_source

with open(f'{output_directory}/journal_english_linguistics_sample11.html', 'w', encoding='utf8', newline='\n') as file:
    file.write(document_page_sample_11)

# Closing the WebDriver
driver.quit()

#### Scraping the articles

##### Function to scrape the articles

In [29]:
def scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge):
    # Setting up the WebDriver
    service = Service(r'C:\Users\eyamr\OneDrive\00-Technology\laelgelc\edgedriver_win64\msedgedriver.exe')
    driver = webdriver.Edge(service=service)

    data = []

    for i in tqdm(range(start_page, end_page + 1)):
        url = f"{article_list_url}{i}"
        driver.get(url)

        # Explicit wait for main article elements
        wait = WebDriverWait(driver, 10)

        # Ensure stable page load
        max_wait_time = 30
        start_time = time.time()
        previous_html = ''
        while True:
            current_html = driver.page_source
            if current_html == previous_html or time.time() - start_time > max_wait_time:
                break
            previous_html = current_html
            time.sleep(2)

        # Parse page source with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        # Extract Volume/Issue
        volume_issue_tag = soup.find('title')
        volume_issue = volume_issue_tag.get_text(strip=True) if volume_issue_tag else ''

        # Extract articles
        articles = soup.find_all('div', class_='issue-item issue-item--toc issue-item--checkbox')

        for article in articles:
            # Extracting the article type
            article_type_tag = article.find('span', string='Research article')
            article_type = article_type_tag.get_text(strip=True) if article_type_tag else ''

            # Extracting the title
            title_tag = article.find('div', class_='issue-item__title').find('h5')
            title = title_tag.get_text(strip=True) if title_tag else ''

            # Extracting the article URL
            article_url_tag = article.find('div', class_='issue-item__title').find('a')
            article_url = root_url + article_url_tag['href'] if article_url_tag else ''

            # Extracting authors
            authors_list = article.find('ul', class_='rlist--inline loa comma')
            authors = ', '.join([author.get_text(strip=True) for author in authors_list.find_all('span')]) if authors_list else ''

            # Extracting published date (removing "First published")
            published_tag = article.find('span', string=lambda text: text and text.startswith("First published"))
            published = published_tag.get_text(strip=True).replace("First published ", "") if published_tag else ''

            # Extracting DOI
            doi_tag = article.find('input', {'name': 'doi'})
            doi = doi_root_url + doi_tag['value'] if doi_tag else ''

            # Extracting Open Access status
            open_access_tag = article.find('span', class_='issue-item-access').find('span')
            open_access = open_access_tag.get_text(strip=True) if open_access_tag else ''

            # Extracting PDF URL (Replacing 'reader' with 'pdf')
            pdf_url_tag = article.find('a', title='download')
            pdf_url = root_url + pdf_url_tag['href'].replace('reader', 'pdf') if pdf_url_tag and pdf_url_tag.get('href') else ''

            # Append extracted data
            data.append({
                'Article Type': article_type,
                'Title': title,
                'URL': article_url,
                'Authors': authors,
                'Vol/Issue': volume_issue,
                'Published': published,
                'DOI': doi,
                'Open Access': open_access,
                'PDF URL': pdf_url,
                'Area of Knowledge': area_of_knowledge
            })

    # Close WebDriver
    driver.quit()

    return pd.DataFrame(data)

##### Volume 48 (Issue 1, March 2020; Issue 2, June 2020; Issue 3, September 2020; Issue 4, December 2020)

In [30]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/enga/48/'
start_page = 1
end_page = 4
area_of_knowledge = 'Linguistic, literature and arts'

In [31]:
df_v48i1i2i3i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 4/4 [00:26<00:00,  6.52s/it]


##### Volume 49 (Issue 1, March 2021; Issue 2, June 2021; Issue 3, September 2021; Issue 4, December 2021)

In [32]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/enga/49/'
start_page = 1
end_page = 4
area_of_knowledge = 'Linguistic, literature and arts'

In [33]:
df_v49i1i2i3i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 4/4 [00:24<00:00,  6.16s/it]


##### Volume 50 (Issue 1, March 2022; Issue 2, June 2022; Issue 3, September 2022; Issue 4, December 2022)

In [34]:
root_url = 'https://journals.sagepub.com'
doi_root_url = 'https://doi.org/'
article_list_url = 'https://journals.sagepub.com/toc/enga/50/'
start_page = 1
end_page = 4
area_of_knowledge = 'Linguistic, literature and arts'

In [35]:
df_v50i1i2i3i4 = scrape_articles(root_url, doi_root_url, article_list_url, start_page, end_page, area_of_knowledge)

100%|██████████| 4/4 [00:25<00:00,  6.45s/it]


##### Concatenating the DataFrames

In [36]:
df_journal_english_linguistics = pd.concat([df_v48i1i2i3i4, df_v49i1i2i3i4, df_v50i1i2i3i4], ignore_index=True)

In [37]:
df_journal_english_linguistics['Published'] = pd.to_datetime(
    df_journal_english_linguistics['Published'], format='%B %d, %Y'
)

In [38]:
df_journal_english_linguistics.dtypes

Article Type                 object
Title                        object
URL                          object
Authors                      object
Vol/Issue                    object
Published            datetime64[ns]
DOI                          object
Open Access                  object
PDF URL                      object
Area of Knowledge            object
dtype: object

##### Selecting the 'Open access' and 'Full text access' articles published before the advent of ChatGPT

In [39]:
unique_open_access = df_journal_english_linguistics['Open Access'].dropna().unique()
unique_open_access

array(['Restricted access', 'Open Access', 'Available access'],
      dtype=object)

In [40]:
df_journal_english_linguistics_open_access = df_journal_english_linguistics[
    (df_journal_english_linguistics['Open Access'] == 'Open Access') | 
    (df_journal_english_linguistics['Open Access'] == 'Available access')
]
df_journal_english_linguistics_open_access = df_journal_english_linguistics_open_access[df_journal_english_linguistics_open_access['Published'] <= '2022-11-30']
df_journal_english_linguistics_open_access = df_journal_english_linguistics_open_access.reset_index(drop=True)

#### Retaining only the `Research articles`

In [41]:
df_journal_english_linguistics_open_access = df_journal_english_linguistics_open_access.loc[
    df_journal_english_linguistics_open_access['Article Type'] == 'Research article'
].reset_index(drop=True)
df_journal_english_linguistics_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,PDF URL,Area of Knowledge
0,Research article,Three Types of Old English Adjectival Postposi...,https://journals.sagepub.com/doi/abs/10.1177/0...,Maciej Grabski,"Journal of English Linguistics - Volume 48, Nu...",2020-04-27,https://doi.org/10.1177/0075424220911067,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
1,Research article,Non-sexist Language Policy and the Rise (and F...,https://journals.sagepub.com/doi/abs/10.1177/0...,Laura L. Paterson,"Journal of English Linguistics - Volume 48, Nu...",2020-07-18,https://doi.org/10.1177/0075424220938949,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
2,Research article,On the History of the English Progressive Cons...,https://journals.sagepub.com/doi/abs/10.1177/0...,Teresa Fanego,"Journal of English Linguistics - Volume 48, Nu...",2020-08-10,https://doi.org/10.1177/0075424220945008,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
3,Research article,ALittleSomething Goes a Long Way:Littlein the ...,https://journals.sagepub.com/doi/abs/10.1177/0...,"Claudia Claridge, Ewa Jonsson, Merja Kytö","Journal of English Linguistics - Volume 49, Nu...",2021-01-15,https://doi.org/10.1177/0075424220982063,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
4,Research article,Intensification in Eighteenth Century Medical ...,https://journals.sagepub.com/doi/abs/10.1177/0...,Turo Hiltunen,"Journal of English Linguistics - Volume 49, Nu...",2021-01-15,https://doi.org/10.1177/0075424220982649,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
5,Research article,Intensificatory Tautology in the History of En...,https://journals.sagepub.com/doi/abs/10.1177/0...,Victorina González-Díaz,"Journal of English Linguistics - Volume 49, Nu...",2021-04-01,https://doi.org/10.1177/0075424221999095,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
6,Research article,Intonation and Referee Design Phenomena in the...,https://journals.sagepub.com/doi/abs/10.1177/0...,Nicole Holliday,"Journal of English Linguistics - Volume 49, Nu...",2021-07-06,https://doi.org/10.1177/00754242211024722,Available access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
7,Research article,nurse Vowels in Scottish Standard English: Sti...,https://journals.sagepub.com/doi/abs/10.1177/0...,"Zeyu Li, Ulrike Gut, Ole Schützler","Journal of English Linguistics - Volume 49, Nu...",2021-07-06,https://doi.org/10.1177/00754242211025586,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
8,Research article,Back Vowel Dynamics and Distinctions in Southe...,https://journals.sagepub.com/doi/abs/10.1177/0...,"Joseph A. Stanley, Margaret E. L. Renwick, Kat...","Journal of English Linguistics - Volume 49, Nu...",2021-12-20,https://doi.org/10.1177/00754242211043163,Available access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"
9,Research article,Mechanisms of Grammaticalization in the Variat...,https://journals.sagepub.com/doi/abs/10.1177/0...,Claire Childs,"Journal of English Linguistics - Volume 49, Nu...",2021-12-20,https://doi.org/10.1177/00754242211044837,Open Access,https://journals.sagepub.com/doi/pdf/10.1177/0...,"Linguistic, literature and arts"


#### Inspecting a few articles

In [42]:
df_journal_english_linguistics_open_access.at[0, 'URL']

'https://journals.sagepub.com/doi/abs/10.1177/0075424220911067'

#### Exporting to a file

In [43]:
df_journal_english_linguistics_open_access.to_json(f"{output_directory}/journal_english_linguistics_open_access.jsonl", orient='records', lines=True)

### Scraping [Modern Language Review](https://www.mhra.org.uk/journals/MLR)

This journal had no 'Open Access' articles at the time of data collection.

### Scraping [PMLA: Publications of the Modern Language Association](https://www.mla.org/Publications/Journals/PMLA)

This journal was not open for the public at the time of data collection.

### Scraping [Corpora](https://www.euppublishing.com/journal/cor)

#### Capturing the data manually

In [4]:
corpora_dict = {
   'Article Type': [
      'Research article',
      'Research article',
      'Research article',
      'Research article',
      'Research article',
      'Research article',
      'Research article',
      'Research article',
      'Research article',
   ],
   'Title': [
      "Sketching women: a corpus-based approach to representations of women's agency in political Internet corpora in Arabic and English",
      "PTPARL-D: an annotated corpus of forty-four years of Portuguese parliamentary debates",
      "English learner corpora and research in Korea",
      "English lexical bundles in a learner corpus of argumentative essays written by Korean university students",
      "The ordering of relative clauses and determiner phrases in Chinese interlanguage: a multifactorial study",
      "Exploring the use of make + noun collocations by Japanese EFL learners through a bilingual essay corpus",
      "Learner corpus research in Hong Kong: past, present and future",
      "A corpus-based study of native speakers’ and Taiwanese EFL learners’ use of the adverb just",
      "Learner corpus research in New Zealand",
   ],
   'URL': [
      'https://www.euppublishing.com/doi/full/10.3366/cor.2020.0184',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2021.0226',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0244',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0245',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0246',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0247',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0248',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0249',
      'https://www.euppublishing.com/doi/full/10.3366/cor.2022.0250',
   ],
   'Authors': [
      'Kamran Karimullah',
      'Paulo Almeida, Manuel Marques-Pita, and Joana Gonçalves-Sá',
      'Heokseung Kwon',
      'Isaiah WonHo Yoo and Yu Kyoung Shin',
      'Jiajin Xu and Zhaoxia Liu',
      'Ryo Sawaguchi and Atsushi Mizumoto',
      'Kanglong Liu, Joyce Oiwun Cheung, and Nan Zhao',
      'Yi-Ching Lin and Siaw-Fong Chung',
      'Anna Siyanova-Chanturia, Jean Parkinson, and Taha Omidian',
   ],
   'Vol/Issue': [
      'Volume 15, Issue 1',
      'Volume 16, Issue 3',
      'Volume 17, Issue Supplement',
      'Volume 17, Issue Supplement',
      'Volume 17, Issue Supplement',
      'Volume 17, Issue Supplement',
      'Volume 17, Issue Supplement',
      'Volume 17, Issue Supplement',
      'Volume 17, Issue Supplement',
   ],
   'Published': [
      '31 March 2020',
      '25 November 2021',
      '27 September 2022',
      '27 September 2022',
      '27 September 2022',
      '27 September 2022',
      '27 September 2022',
      '27 September 2022',
      '27 September 2022',
   ],
   'DOI': [
      'https://doi.org/10.3366/cor.2020.0184',
      'https://doi.org/10.3366/cor.2021.0226',
      'https://doi.org/10.3366/cor.2022.0244',
      'https://doi.org/10.3366/cor.2022.0245',
      'https://doi.org/10.3366/cor.2022.0246',
      'https://doi.org/10.3366/cor.2022.0247',
      'https://doi.org/10.3366/cor.2022.0248',
      'https://doi.org/10.3366/cor.2022.0249',
      'https://doi.org/10.3366/cor.2022.0250',
   ],
   'Open Access': [
      'Open access',
      'Open access',
      'Free access',
      'Free access',
      'Free access',
      'Free access',
      'Free access',
      'Free access',
      'Free access',
   ],
   'PDF URL': [
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2020.0184',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2021.0226',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0244',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0245',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0246',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0247',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0248',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0249',
      'https://www.euppublishing.com/doi/pdf/10.3366/cor.2022.0250',
   ],
   'Discipline': [
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
      'Linguistic, literature and arts',
   ]
}

In [5]:
df_corpora_open_access = pd.DataFrame(corpora_dict)

In [6]:
df_corpora_open_access['Published'] = pd.to_datetime(df_corpora_open_access['Published'], dayfirst=True)

In [7]:
df_corpora_open_access

,Article Type,Title,URL,Authors,Vol/Issue,Published,DOI,Open Access,PDF URL,Discipline
0,Research article,Sketching women: a corpus-based approach to re...,https://www.euppublishing.com/doi/full/10.3366...,Kamran Karimullah,"Volume 15, Issue 1",2020-03-31,https://doi.org/10.3366/cor.2020.0184,Open access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
1,Research article,PTPARL-D: an annotated corpus of forty-four ye...,https://www.euppublishing.com/doi/full/10.3366...,"Paulo Almeida, Manuel Marques-Pita, and Joana ...","Volume 16, Issue 3",2021-11-25,https://doi.org/10.3366/cor.2021.0226,Open access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
2,Research article,English learner corpora and research in Korea,https://www.euppublishing.com/doi/full/10.3366...,Heokseung Kwon,"Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0244,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
3,Research article,English lexical bundles in a learner corpus of...,https://www.euppublishing.com/doi/full/10.3366...,Isaiah WonHo Yoo and Yu Kyoung Shin,"Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0245,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
4,Research article,The ordering of relative clauses and determine...,https://www.euppublishing.com/doi/full/10.3366...,Jiajin Xu and Zhaoxia Liu,"Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0246,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
5,Research article,Exploring the use of make + noun collocations ...,https://www.euppublishing.com/doi/full/10.3366...,Ryo Sawaguchi and Atsushi Mizumoto,"Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0247,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
6,Research article,"Learner corpus research in Hong Kong: past, pr...",https://www.euppublishing.com/doi/full/10.3366...,"Kanglong Liu, Joyce Oiwun Cheung, and Nan Zhao","Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0248,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
7,Research article,A corpus-based study of native speakers’ and T...,https://www.euppublishing.com/doi/full/10.3366...,Yi-Ching Lin and Siaw-Fong Chung,"Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0249,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"
8,Research article,Learner corpus research in New Zealand,https://www.euppublishing.com/doi/full/10.3366...,"Anna Siyanova-Chanturia, Jean Parkinson, and T...","Volume 17, Issue Supplement",2022-09-27,https://doi.org/10.3366/cor.2022.0250,Free access,https://www.euppublishing.com/doi/pdf/10.3366/...,"Linguistic, literature and arts"


In [8]:
df_corpora_open_access.at[0, 'URL']

'https://www.euppublishing.com/doi/full/10.3366/cor.2020.0184'

#### Exporting to a file

In [9]:
df_corpora_open_access.to_json(f"{output_directory}/corpora_open_access.jsonl", orient='records', lines=True)